# Profissionais de dados: O que fazem? Quem são? Como atuam? Onde vivem? Quanto ganham? O que pensam?

![Imagem de header](https://raw.githubusercontent.com/leosimoes/DataHackers-State-of-Data-Brazil-2021/main/StateOfData-2021-Logo.jpeg "Logo State-of-Data")

*Autor: Leonardo Simões*

Data: 12/07/22

---
## Sumário
- [1.Introdução](#intro)
- [2.Objetivos](#obj)
- [3.Preparação dos dados](#data_wrangling) 
    - [3.1.Aquisição dos dados](#gather)
    - [3.2.Avaliação dos dados](#evaluation)
        - [3.2.1.Dimensões dos dataframes](#dim_df)
        - [3.2.2.Nomes e tipos de colunas](#names_types_columns)
        - [3.2.3.Colunas repetidas](#repeated_columns)
        - [3.2.4.Valores ausentes](#values_na)
        - [3.2.5.Verificando valores](#verify_values)
        - [3.2.6.Problemas identificados](#problems_identified)
    - [3.3.Limpeza e organização dos dados](#clean_and_tidiness)
        - [3.3.1.Remover colunas desnecessárias](#dropna_rows)
        - [3.3.2.Remover linhas com valores ausentes em algumas colunas](#rename_column)
        - [3.3.3.Alterar tipos das colunas](#change_column_type)
        - [3.3.4.Preencher das colunas de texto](#fillna_string_columns)
        - [3.3.5.Formatar valores](#fix_values)
        - [3.3.6.Salvar dataframe em um arquivo .csv](#save_csv)
- [4.Análise dos dados](#eda)
    - [4.1.Pontos gerais da pesquisa](#pesquisa_geral)
    - [4.2.Engenheiro de Dados](#engenheiro_dados)
    - [4.3.Analista de BI](#analista_bi)
    - [4.4.Analista de Dados](#analista_dados)
    - [4.5.Cientista de Dados](#cientista_dados) 
- [5.Aprendizado de máquina](#pre_processing)
    - [5.1.Pré-Processamento](#pre_processing)
    - [5.2.Treinamento do modelo](#model_train)
    - [5.3.Predição](#predictions)
- [Referências](#references)

---
<a id='intro'></a>
## 1. Introdução

Este projeto mostra uma análise dos dados de profissionais que responderam à pesquisa State of Data Brazil em 2021, e serve como meio de participação da competição do Kaggle relacionada a estes dados.

Algumas características e observações deste trabalho são:

- O foco é a análise dos dados, ao invés de predições, como costuma ocorrer na maioria das competições do Kaggle.
- As funções serão definidas logo antes de serem usadas pela primeira vez, possibilitando a visão de sua execução em seguida.
- A análise em questão não foi feita para exaurir a maioria (ou todos) os dados da pesquisa, como foi feito pelo relatório oficial divulgado.
- As imagens e gráficos usados buscam facilitar o entendimento dos dados e informações.
- A reprodutibilidade do notebook depende que os outros arquivos usados nele estejam nos diretórios correspondentes.

O fluxo de análise de dados adotado neste projeto foi definido pelas etapas de preparação dos dados (aquisição, avaliação, limpeza, análise dos dados(análise exploratória e visualização) e aprendizado de máquina.

![Imagem de FluxoDoProjeto](Diagrama-Fluxo.png "FluxoDoProjeto img")

---
<a id='obj'></a>
## 2. Objetivos

Os objetivos deste projeto são:

    - Carregar o conjunto de dados da pesquisa State of Data Brazil 2021.
    - Avaliar o conjunto de dados  e identificar problemas.
    - Limpar o conjunto de dados e salvá-lo em um arquivo .csv.
    - Analisar os dados e descobrir informações.
    - Usar gráficos, imagens e técnicas de storytelling para ilustrar as informações e descobertas.
    - Se diferenciar do relatório original do State of Data.
    - Gerar um modelo de aprendizado de máquina de classificação para predições e explicações.
    - Exportar o modelo de aprendizado de máquina gerado.
    - Salvar os dados das predições em um arquivo .csv.

----
<a id='data_wrangling'></a>
## 3. Preparação dos dados

A etapa de preparação dos dados antecede a de análise, e consiste no agrupamentos das etapas de aquisição, avaliação e limpeza dos dados. Os dados são preparados para a análise, ou seja, são carregados, avaliados, observados, organizados, corrigidos e limpos.

In [1]:
# Verificação da versão do Python
from platform import python_version
print('Versão do Python: ', python_version())

Versão do Python:  3.11.2


In [2]:
# Instalação das bibliotecas mais usadas
#!pip install numpy>=1.21 
#!pip install pandas==1.3.5
#!pip install matplotlib==3.5.2
#!pip install seaborn==0.11.2
#!pip install plotly
#!pip install scikit-learn

In [3]:
# Imports das bibliotecas mais usadas
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl


%matplotlib inline

In [4]:
import os
import unidecode
import re

In [5]:
# Imports da biblioteca Plotly
import plotly
import plotly.offline as py
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

py.init_notebook_mode(connected=True)

In [6]:
# Import das bibliotecas usadas no aprendizado de máquina
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

import joblib

In [7]:
# Versões das principais bibliotecas usadas 
print('Versões das bibliotecas')
print('Numpy: ' + np.__version__)
print('Pandas: ' + pd.__version__)
print('Matplotlib: ' + mpl.__version__)
print('Seaborn: ' + sns.__version__)
print('Plotly: ' + plotly.__version__)
print('Sklearn: ' + skl.__version__)

Versões das bibliotecas
Numpy: 1.23.5
Pandas: 1.5.3
Matplotlib: 3.7.0
Seaborn: 0.12.2
Plotly: 5.13.1
Sklearn: 1.2.2


In [8]:
# Exibir todas as linhas do dataframe
pd.set_option("display.max_rows", None)

# Exibir todas as colunas do datraframe
pd.set_option('display.max_columns', None)

#Exibir valores longos de string
pd.set_option('display.max_colwidth', None)

In [9]:
# Definição de valor de semente
RANDOM_STATE = 42

# Definição de temas dos gráficos
# TEMAS = ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]
TEMA_BARRAS = 'plotly_white'
TEMA_BOX = 'plotly_white'
TEMA_TREEMAP = 'ggplot2'
TEMA_PIZZA = 'ggplot2'

---
<a id='gather'></a>
## 3.1. Aquisição dos dados

Nesta etapa os dados da pesquisa State of Data Brazil 2021 são carregados em um dataframe. 

In [10]:
# Definição da função diretorios_arquivos_kaggle()
def diretorios_arquivos_kaggle():
    """
    Imprime os diretórios dos arquivos presentes
    """
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

In [11]:
# Exibe os diretórios dos arquivos
#diretorios_arquivos_kaggle()

In [12]:
# Definição da função "carregar_dados"
def carregar_dados(diretorio_do_arquivo):
    """
    Carrega os dados do State of Date em um dataframe.
    Formata o nome das colunas do dataframe.
    Remove linhas duplicadas no dataframe.
    Retorna um DataFrame
    
    Argumento:
    diretorio_do_arquivo -- o diretório do arquivo arquivo de csv.
    """
    df = pd.read_csv(diretorio_do_arquivo, low_memory=False)
    renomear_colunas = lambda x: x.split("',")[1][2:-2]
    df.rename(renomear_colunas, axis='columns', inplace=True)
    df.drop_duplicates(keep='first', inplace=True)
    
    return df

In [13]:
# Diretório do arquivo usado
# DIRETORIO_ARQUIVO = '/kaggle/input/state-of-data-2021/State of Data 2021 - Dataset - Pgina1.csv'
# DIRETORIO_ARQUIVO = '../input/state-of-data-2021/State of Data 2021 - Dataset - Pgina1.csv'
DIRETORIO_ARQUIVO = 'State of Data 2021 - Dataset - Pgina1.csv'

In [14]:
# Cria dataframe com os dados em arquivo
df_original = carregar_dados(DIRETORIO_ARQUIVO)

# Exibe as 2 primeiras linhas do dataframe
df_original.head(2)

,id,Idade,Faixa idade,Genero,Estado onde mora,uf onde mora,Regiao onde mora,Regiao de origem,Mudou de Estado?,Nivel de Ensino,Área de Formação,Qual sua situação atual de trabalho?,Setor,Numero de Funcionarios,Gestor?,Cargo como Gestor,Cargo Atual,Nivel,Faixa salarial,Quanto tempo de experiência na área de dados você tem?,Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?,Você está satisfeito na sua empresa atual?,Qual o principal motivo da sua insatisfação com a empresa atual?,Falta de oportunidade de crescimento no emprego atual,Salário atual não corresponde ao mercado,Não tenho uma boa relação com meu líder/gestor,Gostaria de trabalhar em em outra área de atuação,Gostaria de receber mais benefícios,O clima de trabalho/ambiente não é bom,Falta de maturidade analítica na empresa,Você participou de entrevistas de emprego nos últimos 6 meses?,Você pretende mudar de emprego nos próximos 6 meses?,Quais os principais critérios que você leva em consideração no momento de decidir onde trabalhar?,Remuneração/Salário,Benefícios,Propósito do trabalho e da empresa,Flexibilidade de trabalho remoto,Ambiente e clima de trabalho,Oportunidade de aprendizado e trabalhar com referências na área,Plano de carreira e oportunidades de crescimento profissional,Maturidade da empresa em termos de tecnologia e dados,Qualidade dos gestores e líderes,Reputação que a empresa tem no mercado,Atualmente qual a sua forma de trabalho?,Qual a forma de trabalho ideal para você?,Caso sua empresa decida pelo modelo 100% presencial qual será sua atitude?,Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?,Quais desses papéis/cargos fazem parte do time (ou chapter) de dados da sua empresa?,Analytics Engineer,Engenharia de Dados/Data Engineer,Analista de Dados/Data Analyst,Cientista de Dados/Data Scientist,Database Administrator/DBA,Analista de Business Intelligence/BI,Arquiteto de Dados/Data Architect,Data Product Manager/DPM,Business Analyst,Quais dessas responsabilidades fazem parte da sua rotina atual de trabalho como gestor?,Pensar na visão de longo prazo de dados da empresa e fortalecimento da cultura analítica da companhia.,Organização de treinamentos e iniciativas com o objetivo de aumentar a maturidade analítica das áreas de negócios.,"Atração, seleção e contratação de talentos para o time de dados.",Decisão sobre contratação de ferramentas e tecnologias relacionadas a dados.,"Sou gestor da equipe responsável pela engenharia de dados e por manter o Data Lake da empresa como fonte única dos dados, garantindo a qualidade e confiabilidade da informação.","Sou gestor da equipe responsável pela entrega de dados, estudos, relatórios e dashboards para as áreas de negócio da empresa.",Sou gestor da equipe responsável por iniciativas e projetos envolvendo Inteligência Artificial e Machine Learning.,"Apesar de ser gestor ainda atuo na parte técnica, construindo soluções/análises/modelos etc.","Gestão de projetos de dados, cuidando das etapas, equipes envolvidas, atingimento dos objetivos etc.","Gestão de produtos de dados, cuidando da visão dos produtos, backlog, feedback de usuários etc.","Gestão de pessoas, apoio no desenvolvimento das pessoas, evolução de carreira",Quais são os 3 maiores desafios que você tem como gestor no atual momento?,a Contratar novos talentos.,b Reter talentos.,c Convencer a empresa a aumentar os investimentos na área de dados.,d Gestão de equipes no ambiente remoto.,e Gestão de projetos envolvendo áreas multidisciplinares da empresa.,f Organizar as informações e garantir a qualidade e confiabilidade.,g Conseguir processar e armazenar um alto volume de dados.,h Conseguir gerar valor para as áreas de negócios através de estudos e experimentos.,i Desenvolver e manter modelos Machine Learning em produção.,j Gerenciar a expectativa das áreas de negócio em relação as entregas das equipes de dados.,"k Garantir a manutenção dos projetos e modelos em produ

In [15]:
# Atribui um nome ao dataframe
NOME_DF_ORIGINAL_2021 = 'State of Data 2021 - Original'

In [16]:
# Verifica que não há linhas duplicadas no dataframe
assert df_original.duplicated().sum() == 0

---
<a id='evaluation'></a>
## 3.2. Avaliação dos dados

Nesta etapa, o dataframe é examinado em busca de problemas de organização e qualidade dos dados. Aqui é feita um tipo de análise focada no formato dos dados, e não em extrair informações sobre a pesquisa em si.

<a id='dim_df'></a>
### 3.2.1. Dimensões do dataframe

In [17]:
# Definição da função print_dimensoes_dataframe()
def print_dimensoes_dataframe(df, nome=''):
    """
    Exibe em uma linha o nome informado e os números de linhas e colunas de df.
    
    Argumentos:
    df -- DataFrame
    nome -- string (Opcional - Valor padrão é '')
    """
    print('As dimensões do dataframe ' + nome +  ' são ' + str(df.shape[0]) + ' linhas e ' + str(df.shape[1]) + ' colunas.')

In [18]:
#exibir_dimensoes
print_dimensoes_dataframe(df_original, nome=NOME_DF_ORIGINAL_2021)

As dimensões do dataframe State of Data 2021 - Original são 2641 linhas e 356 colunas.


<a id='names_types_columns'></a>
### 3.2.2. Nomes e tipos de colunas

In [19]:
# Definição da função calcular_quantidade_de_colunas_por_tipo()
def calcular_quantidade_de_colunas_por_tipo(df):
    """
    Exibe a quantidade de colunas do DataFrame por tipo.
    
    Argumento:
    df -- DataFrame
    """
    tipos = df.dtypes.value_counts().to_frame()
    tipos = tipos.reset_index()
    tipos.columns = ['Tipo de dado', 'Quantidade de Colunas']
    
    return tipos

In [20]:
calcular_quantidade_de_colunas_por_tipo(df_original)

,Tipo de dado,Quantidade de Colunas
0,float64,293
1,object,56
2,int64,7


In [21]:
# Definição da função print_nomes_colunas_por_tipo()
def print_nomes_colunas_por_tipo(df, tipo):
    """
    Exibe as colunas do DataFrame e do tipo informado enumeradas a partir de 1.
    
    Argumento:
    df -- DataFrame
    tipo -- string
    """
    colunas_df = df.select_dtypes(include=tipo).columns.to_list()
    print('As colunas do tipo ' + tipo + ' do dataframe são:')
    for num, coluna in enumerate(colunas_df, start=1):
        print(str(num) + ' - ' + coluna)

In [22]:
# Exibe os nomes das colunas do tipo 'int64'
print_nomes_colunas_por_tipo(df_original, tipo='int64')

As colunas do tipo int64 do dataframe são:
1 - Mudou de Estado?
2 - Blog/Medium do Data Hackers
3 - Podcast do Data Hackers
4 - Newsletter Semanal
5 - Canal do Slack
6 - Canal do Youtube do Data Hackers
7 - Ainda não conhecia o Data Hackers


In [23]:
# Exibe os nomes das colunas do tipo 'float64'
print_nomes_colunas_por_tipo(df_original, tipo='float64')

As colunas do tipo float64 do dataframe são:
1 - Idade
2 - Gestor?
3 - Você está satisfeito na sua empresa atual?
4 - Falta de oportunidade de crescimento no emprego atual
5 - Salário atual não corresponde ao mercado
6 - Não tenho uma boa relação com meu líder/gestor
7 - Gostaria de trabalhar em em outra área de atuação
8 - Gostaria de receber mais benefícios
9 - O clima de trabalho/ambiente não é bom
10 - Falta de maturidade analítica na empresa
11 - Remuneração/Salário
12 - Benefícios
13 - Propósito do trabalho e da empresa
14 - Flexibilidade de trabalho remoto
15 - Ambiente e clima de trabalho
16 - Oportunidade de aprendizado e trabalhar com referências na área
17 - Plano de carreira e oportunidades de crescimento profissional
18 - Maturidade da empresa em termos de tecnologia e dados
19 - Qualidade dos gestores e líderes
20 - Reputação que a empresa tem no mercado
21 - Analytics Engineer
22 - Engenharia de Dados/Data Engineer
23 - Analista de Dados/Data Analyst
24 - Cientista de Da

In [24]:
# Exibe os nomes das colunas do tipo 'object' (strings)
print_nomes_colunas_por_tipo(df_original, tipo='object')

As colunas do tipo object do dataframe são:
1 - id
2 - Faixa idade
3 - Genero
4 - Estado onde mora
5 - uf onde mora
6 - Regiao onde mora
7 - Regiao de origem
8 - Nivel de Ensino
9 - Área de Formação
10 - Qual sua situação atual de trabalho?
11 - Setor
12 - Numero de Funcionarios
13 - Cargo como Gestor
14 - Cargo Atual
15 - Nivel
16 - Faixa salarial
17 - Quanto tempo de experiência na área de dados você tem?
18 - Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?
19 - Qual o principal motivo da sua insatisfação com a empresa atual?
20 - Você participou de entrevistas de emprego nos últimos 6 meses?
21 - Você pretende mudar de emprego nos próximos 6 meses?
22 - Quais os principais critérios que você leva em consideração no momento de decidir onde trabalhar?
23 - Atualmente qual a sua forma de trabalho?
24 - Qual a forma de trabalho ideal para você?
25 - Caso sua empresa decida pelo modelo 100% presencial qual será sua

<a id='repeated_columns'></a>
### 3.2.3. Colunas repetidas

In [25]:
# Definição da função possui_nomes_colunas_repetidos()
def possui_nomes_colunas_repetidos(df, nome=''):
    """
    Informa se o df não possui colunas com nomes repetidos ou a quantidade destes nomes.
    
    Argumento:
    df - DataFrame
    nome -- string (Opcional - Valor padrão é '')
    """
    colunas_nomes, colunas_contagem = np.unique(df.columns.values, return_counts=True)
    quantidade_colunas_repetidas = colunas_nomes[colunas_contagem>1].size
    
    if quantidade_colunas_repetidas != 0:
        print('O dataframe {} possui {} nomes repetidos de colunas.'.format(nome, quantidade_colunas_repetidas))
    else:
        print('O dataframe {} não possui nomes repetidos de colunas.'.format(nome))

In [26]:
# Exibe a quantidade de colunas repetidas
possui_nomes_colunas_repetidos(df_original, NOME_DF_ORIGINAL_2021)

O dataframe State of Data 2021 - Original possui 39 nomes repetidos de colunas.


In [27]:
# Definição da função print_nomes_repetidos_colunas()
def print_nomes_repetidos_colunas(df, nome=''):
    """
    Exibe os nomes repetidos de colunas do DataFrame enumerados a partir de 1.
    
    Argumento:
    df -- DataFrame
    nome -- string (Opcional - Valor padrão é '')
    """
    colunas_nomes, colunas_contagem = np.unique(df.columns.values, return_counts=True)
    nome_repetidos = colunas_nomes[colunas_contagem>1].tolist()
    print('Os nomes repetidos de colunas do dataframe {} são: '.format(nome))
    for num, coluna in enumerate(nome_repetidos, start=1):
        print(str(num) + ' - ' + coluna)

In [28]:
# Exibe os nomes de colunas repetidas enumerados
print_nomes_repetidos_colunas(df_original, NOME_DF_ORIGINAL_2021)

Os nomes repetidos de colunas do dataframe State of Data 2021 - Original são: 
1 - .NET
2 - AWS Glue
3 - Apache Airflow
4 - C/C++/C#
5 - Dados armazenados em bancos NoSQL
6 - Dados georeferenciados
7 - Dados relacionais (estruturados em bancos SQL)
8 - Fivetran
9 - Google Dataflow
10 - IBM DataStage
11 - Imagens
12 - Java
13 - Javascript
14 - Julia
15 - Knime
16 - Luigi
17 - Matlab
18 - Não utilizo ferramentas de ETL
19 - Não utilizo nenhuma linguagem
20 - Oracle Data Integrator
21 - PHP
22 - Planilhas
23 - Python
24 - Qlik Sense
25 - R 
26 - SAP BW ETL
27 - SAS Data Integration
28 - SAS/Stata
29 - SQL
30 - SQL & Stored Procedures
31 - SQL Server Integration Services (SSIS)
32 - Scala
33 - Scripts Python
34 - Stitch
35 - Talend
36 - Textos/Documentos
37 - Visual Basic/VBA
38 - Vídeos
39 - Áudios


In [29]:
# Verificando um caso de nome repetido de colunas
df_original['Dados relacionais (estruturados em bancos SQL)'].head()

,Dados relacionais (estruturados em bancos SQL),Dados relacionais (estruturados em bancos SQL)
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


<a id='values_na'></a>
### 3.2.4. Valores ausentes

In [30]:
# Definição da função print_quantidade_valores_ausentes()
def print_quantidade_valores_ausentes(df):
    """
    Exibe a quantidade de valores ausentes no dataframe.
    
    Argumento:
    df -- DataFrame
    """
    quantidade_valores_ausentes = df.isna().sum().sum()
    print('A quantidade total de valores ausentes é ' + str(quantidade_valores_ausentes) + '.')

In [31]:
# Exibe a quantidade de valores ausentes no dataframe.
print_quantidade_valores_ausentes(df_original)

A quantidade total de valores ausentes é 517055.


In [32]:
# Definição da função print_quantidade_colunas_com_valores_ausentes()
def print_quantidade_colunas_com_valores_ausentes(df):
    """
    Exibe a quantidade de colunas com valores ausentes.
    
    Argumento:
    df -- DataFrame
    """
    valores_na = df.isna().sum()
    valores_na = valores_na[valores_na > 0]

    print('A quantidade de colunas com valores ausentes é ' + str(valores_na.shape[0]) + '.')

In [33]:
# Exibe a quantidade de colunas com valores ausentes.
print_quantidade_colunas_com_valores_ausentes(df_original)

A quantidade de colunas com valores ausentes é 341.


In [34]:
# Definição da função print_quantidade_colunas_sem_valores_ausentes()
def print_quantidade_colunas_sem_valores_ausentes(df):
    """
    Exibe a quantidade de colunas sem valores ausentes.
    
    Argumento:
    df -- DataFrame
    """
    valores_nna = df.isna().sum()
    valores_nna = valores_nna[valores_nna == 0]
    print('A quantidade de colunas sem valores ausentes é ' + str(valores_nna.shape[0]) + '.')

In [35]:
# Exibe a quantidade de colunas sem valores ausentes.
print_quantidade_colunas_sem_valores_ausentes(df_original)

A quantidade de colunas sem valores ausentes é 15.


In [36]:
# Definição da função exibir_valores_ausentes_colunas()
def exibir_valores_ausentes_colunas(df):
    """
    Crie e exibe um dataframe que indica o nome, o tipo, quantidade e porcentagem de valores ausentes 
    para cada coluna com valores ausentes.
    
    Argumento:
    df -- DataFrame
    """
    valores_na = df.isna().sum()
       
    informacoes = pd.DataFrame({'Coluna': df.columns,
                                'Tipo': df.dtypes,
                                'Quantidade de ausentes': valores_na.values,
                                'Porcentagem de ausentes': valores_na / df.shape[0]
                                })
    
    informacoes.reset_index(drop=True, inplace=True)
    informacoes['Porcentagem de ausentes'] = (informacoes['Porcentagem de ausentes'] * 100).round(2) 
    informacoes['Porcentagem de ausentes'] = informacoes['Porcentagem de ausentes'].astype(str) + ' %'
    
    return informacoes

In [37]:
# Exibe valores ausentes por coluna
exibir_valores_ausentes_colunas(df_original)

,Coluna,Tipo,Quantidade de ausentes,Porcentagem de ausentes
0,id,object,0,0.0 %
1,Idade,float64,28,1.06 %
2,Faixa idade,object,0,0.0 %
3,Genero,object,0,0.0 %
4,Estado onde mora,object,36,1.36 %
5,uf onde mora,object,36,1.36 %
6,Regiao onde mora,object,0,0.0 %
7,Regiao de origem,object,2158,81.71 %
8,Mudou de Estado?,int64,0,0.0 %
9,Nivel de Ensino,object,0,0.0 %


In [38]:
# Definição da função exibir_quantidades_valores_ausentes_colunas()
def exibir_quantidades_valores_ausentes_colunas(df):
    """
    Crie e exibe um dataframe que indica a quantidade de colunas para cada quantidade de valores ausentes.
    
    Argumento:
    df -- DataFrame
    """
    valores_na = df.isna().sum()
    df_na = valores_na.value_counts().to_frame().reset_index(level=0)
    df_na.columns = ['Quantidade de valores ausentes', 'Quantidade de colunas']
    df_na = df_na.sort_values(by=['Quantidade de valores ausentes'], ascending=False)
    df_na = df_na.reset_index(drop=True)
    
    return df_na

In [39]:
# Exibir quantidades valores ausentes por quantidade de colunas
exibir_quantidades_valores_ausentes_colunas(df_original)

,Quantidade de valores ausentes,Quantidade de colunas
0,2484,1
1,2482,1
2,2480,1
3,2433,14
4,2344,1
5,2333,1
6,2240,2
7,2238,37
8,2231,47
9,2163,1


<a id='verify_values'></a>
### 3.2.5. Verificando valores

In [40]:
# Definição da função contar_valores()
def contar_valores(serie):
    """
    Retorna um dataframe com a quantidade e porcentagem de cada valor em serie.
    
    Argumento:
    serie - Serie
    """
    quantidade = serie.value_counts()
    percentagem = serie.value_counts(normalize=True)
    percentagem = (percentagem * 100).round(2)
    contagem_percentagem = pd.concat([quantidade, percentagem], axis=1)
    contagem_percentagem.columns = ['Quantidade', 'Porcentagem (%)']
    
    return contagem_percentagem

In [41]:
# Verifica os valores para a coluna 'Faixa Salarial'
contar_valores(df_original['Faixa salarial'])

,Quantidade,Porcentagem (%)
de R$ 8.001/mês a R$ 12.000/mês,477,20.20
de R$ 4.001/mês a R$ 6.000/mês,402,17.03
de R$ 6.001/mês a R$ 8.000/mês,393,16.65
de R$ 12.001/mês a R$ 16.000/mês,262,11.10
de R$ 3.001/mês a R$ 4.000/mês,183,7.75
de R$ 2.001/mês a R$ 3000/mês,180,7.62
de R$ 16.001/mês a R$ 20.000/mês,129,5.46
de R$ 1.001/mês a R$ 2.000/mês,126,5.34
de R$ 20.001/mês a R$ 25.000/mês,62,2.63
de R$ 25.001/mês a R$ 30.000/mês,43,1.82


In [42]:
# Verifica os valores para a coluna 'Atuacao'
contar_valores(df_original['Atuacao'])

,Quantidade,Porcentagem (%)
Análise de Dados,849,32.15
Gestor,507,19.20
Ciência de Dados,410,15.52
Engenharia de Dados,403,15.26
Buscando emprego na área de dados.,244,9.24
Outra,228,8.63


In [43]:
# Verifica os valores para a coluna 'Nivel'
contar_valores(df_original['Nivel'])

,Quantidade,Porcentagem (%)
Pleno,657,35.44
Júnior,621,33.50
Sênior,576,31.07


In [44]:
# Verifica os valores para a coluna 'Nivel de ensino'
contar_valores(df_original['Nivel de Ensino'])

,Quantidade,Porcentagem (%)
Graduação/Bacharelado,908,34.38
Pós-graduação,823,31.16
Estudante de Graduação,389,14.73
Mestrado,342,12.95
Doutorado ou Phd,115,4.35
Não tenho graduação formal,61,2.31
Prefiro não informar,3,0.11


In [45]:
# Verifica os valores para a coluna 'Cargo Atual'
contar_valores(df_original['Cargo Atual'])

,Quantidade,Porcentagem (%)
Cientista de Dados/Data Scientist,356,19.20
Analista de BI/BI Analyst/Analytics Engineer,338,18.23
Analista de Dados/Data Analyst,324,17.48
Engenheiro de Dados/Data Engineer,300,16.18
Outro,113,6.09
Analista de Negócios/Business Analyst,96,5.18
Desenvolvedor ou Engenheiro de Software,91,4.91
Engenheiro de Machine Learning/ML Engineer,47,2.54
Outras Engenharias (não inclui dev),33,1.78
Estatístico,25,1.35


In [46]:
# Verifica os valores para a coluna 'Regiao onde mora'
contar_valores(df_original['Regiao onde mora'])

,Quantidade,Porcentagem (%)
Sudeste,1670,63.23
Sul,400,15.15
Nordeste,299,11.32
Centro-oeste,183,6.93
Exterior,53,2.01
Norte,36,1.36


In [47]:
# Verifica os valores para a coluna 'Estado onde mora'
contar_valores(df_original['Estado onde mora'])

,Quantidade,Porcentagem (%)
São Paulo (SP),1076,41.31
Minas Gerais (MG),340,13.05
Rio de Janeiro (RJ),222,8.52
Paraná (PR),149,5.72
Rio Grande do Sul (RS),136,5.22
Santa Catarina (SC),115,4.41
Distrito Federal (DF),96,3.69
Ceará (CE),68,2.61
Pernambuco (PE),58,2.23
Bahia (BA),58,2.23


In [48]:
# Verifica os valores para a coluna 'Uf onde mora'
contar_valores(df_original['uf onde mora'])

,Quantidade,Porcentagem (%)
SP,1076,41.31
MG,340,13.05
RJ,222,8.52
PR,149,5.72
RS,136,5.22
SC,115,4.41
DF,96,3.69
CE,68,2.61
PE,58,2.23
BA,58,2.23


In [49]:
# Verifica os valores para a coluna 'Quais das iniciativas do Data Hackers que você já acessou/acompanhou?'
contar_valores(df_original['Quais das iniciativas do Data Hackers que você já acessou/acompanhou?'])

,Quantidade,Porcentagem (%)
Ainda não conhecia o Data Hackers,679,25.71
"Blog/Medium do Data Hackers, Podcast do Data Hackers",160,6.06
Podcast do Data Hackers,151,5.72
"Blog/Medium do Data Hackers, Podcast do Data Hackers, Newsletter Semanal, Canal do Slack, Canal do Youtube do Data Hackers",143,5.41
"Blog/Medium do Data Hackers, Podcast do Data Hackers, Newsletter Semanal, Canal do Slack",139,5.26
Blog/Medium do Data Hackers,139,5.26
"Blog/Medium do Data Hackers, Podcast do Data Hackers, Newsletter Semanal",129,4.88
"Blog/Medium do Data Hackers, Podcast do Data Hackers, Canal do Slack",91,3.45
Newsletter Semanal,62,2.35
"Blog/Medium do Data Hackers, Podcast do Data Hackers, Newsletter Semanal, Canal do Youtube do Data Hackers",60,2.27


<a id='problems_identified'></a>
### 3.2.6. Problemas identificados

Problemas identificados na avaliação dos dados:

- Há muitas colunas no dataframe, além do necessário. Muitas colunas foram geradas ao extrair respostas de alguma outra coluna.
- Há colunas repetidas, já que algumas colunas diferentes possuem algumas respostas iguais.
- A maioria das colunas numéricas possuem valores 0 ("Falso") e 1 ("Verdadeiro"), e mesmo assim possuem muitos valores ausentes.
- A informação de ser gestor está presente nas colunas 'Área de Atuacao', 'Gestor?, 'Cargo como Gestor' e não em 'Cargo Atual'.
- Há valores com espaços em branco no início e/ou final do valor.
- Na coluna 'Cargo Atual' há 'Arquiteto de Dados' e 'Arquiteto de dados'.
- Os valores de 'Faixa salarial' apresentam um problema de exibição devido a presença de dois caracteres '$'.

---
<a id='clean_and_tidiness'></a>
## 3.3 Limpeza e organização dos dados

Nesta etapa, os problemas de organização e qualidade dos dados do dataframe serão resolvidos.

In [50]:
# Copia o dataframe para realizar as operações de limpeza
df_final = df_original.copy()
NOME_DF_FINAL_2021 = 'State of Data 2021 - Final'

<a id='drop_columns'></a>
### 3.3.1 Remover colunas desnecessárias

Remover as colunas numéricas, exceto "Idade" e "Você está satisfeito na sua empresa atual?". 

Remover a coluna 'Regiao de origem'.

In [51]:
# Verifica quais indivíduos possuem cargo de 'Gestor'
is_cargo_gestor = df_final['Cargo como Gestor'].notnull() & df_final['Cargo Atual'].isnull()

In [52]:
# Define o 'Cargo Atual' a partir de 'Cargo como Gestor'
df_final.loc[is_cargo_gestor, 'Cargo Atual'] = df_final.loc[is_cargo_gestor, 'Cargo como Gestor']

In [53]:
# Define quais colunas numéricas serão mantidas
colunas_numericas_mantidas = np.array(['Idade','Você está satisfeito na sua empresa atual?']).astype('object')

In [54]:
# Define quais colunas do tipo object (string) serão mantidas
colunas_string = df_final.select_dtypes(include=['object']).columns.values
colunas_string = colunas_string[colunas_string != 'Regiao de origem']

In [55]:
# Define quais colunas serão mantidas
colunas_df_final = np.concatenate((colunas_numericas_mantidas, colunas_string), axis=None)
colunas_df_final = colunas_df_final[colunas_df_final!='Cargo como Gestor']

In [56]:
# Exclui colunas indesejadas do df_final
df_final = df_final[colunas_df_final]

In [57]:
# Verifica se não há colunas com nomes repetidos
possui_nomes_colunas_repetidos(df_final, NOME_DF_FINAL_2021)

O dataframe State of Data 2021 - Final não possui nomes repetidos de colunas.


In [58]:
# Verifica se as colunas do df_final são as escolhidas
assert (colunas_df_final == df_final.columns.values).all()

In [59]:
# Verifica se a coluna 'Regiao de origem' foi removida do df_final
assert 'Regiao de origem' not in df_final.columns.values

In [60]:
# Verifica se a coluna 'Cargo como Gestor' foi removida do df_final
assert 'Cargo como Gestor' not in df_final.columns.values

<a id='dropna_rows'></a>
### 3.3.2 Remover linhas com valores ausentes em algumas colunas

As linhas com valores ausentes para 'Idade', 'Estado onde mora', 'uf onde mora' ou 'Faixa salarial' são removidas.

In [61]:
# Define para quais colunas com valores ausentes terão linhas removidas
colunas_para_dropar_valores_ausentes = ['Idade', 'Estado onde mora', 'uf onde mora', 'Faixa salarial']

In [62]:
# Remove linhas com valores ausentes em alguma das colunas informadas
df_final.dropna(subset=colunas_para_dropar_valores_ausentes, inplace=True)

In [63]:
# Definição da função calcular_quantidade_valores_ausentes()
def calcular_quantidade_valores_ausentes(df):
    """
    Retorna a quantidade de valores ausentes no dataframe
    
    Argumento:
    df -- DataFrame
    """
    return df.isna().sum().sum()

In [64]:
# Verifica se não há colunas com valores ausentes
assert calcular_quantidade_valores_ausentes(df_final[colunas_para_dropar_valores_ausentes]) == 0

<a id='change_column_type'></a>
### 3.3.3 Alterar tipos das colunas

A coluna 'Idade' é convertida para o tipo int64 (inteiro).

In [65]:
# Converte a coluna 'Idade' para int64 (inteiro)
df_final['Idade'] = df_final['Idade'].astype('int64', copy=False)

In [66]:
# Verifica se a coluna 'Idade' é do tipo int64 (inteiro)
assert df_final['Idade'].dtype == np.dtype('int64')

A coluna "Você está satisfeito na sua empresa atual?" é convertida para o tipo string (object).

In [67]:
# Converte a coluna 'Você está satisfeito na sua empresa atual?' para object (string)
COLUNA_SATISFEITO = 'Você está satisfeito na sua empresa atual?'
df_final[COLUNA_SATISFEITO] = df_final[COLUNA_SATISFEITO].astype(str, copy=False)
df_final.loc[df_final[COLUNA_SATISFEITO] == '1.0', COLUNA_SATISFEITO] = 'Sim'
df_final.loc[df_final[COLUNA_SATISFEITO] == '0.0', COLUNA_SATISFEITO] = 'Não'

In [68]:
# Verifica se a coluna 'Você está satisfeito na sua empresa atual?' é do tipo object (string)
assert df_final[COLUNA_SATISFEITO].dtype == np.dtype('object')

<a id='fillna_string_columns'></a>
### 3.3.4 Preencher das colunas de texto

Os valores ausentes das colunas do tipo string (object) são preenchidos com "Não Informado".

In [69]:
# Define o valor default para preencher valores ausente do tipo object (string) 
VALOR_STRING = 'Não Informado'

In [70]:
# Identifica os nomes das colunas que são do tipo object (string)
colunas_string = df_final.select_dtypes(include=['object']).columns.values

In [71]:
# Definição da função preencher_valores_ausentes()
def preencher_valores_ausentes(df, colunas, valor):
    """
    Retorna um dataframe cujos valores ausentes foram preenchidos com o valor fornecido.
    
    Argumentos:
    df -- DataFrame
    colunas -- Lista de string
    valor -- mesmo tipo de df[coluna]
    """
    return df.loc[:, colunas].fillna(valor)

In [72]:
# Preenche os valores ausentes das colunas do tipo object (string)
df_final[colunas_string] = preencher_valores_ausentes(df_final, colunas_string, VALOR_STRING)

In [73]:
# Verifica se não há valores ausentes no dataframe
assert calcular_quantidade_valores_ausentes(df_final[colunas_string]) == 0

<a id='fix_values'></a>
### 3.3.5. Formatar valores

Os valores das faixas salariais são formatados corrigindo o espaçamento e inserindo uma letra maiúscula entre parênteses ao início, possibilitando a ordenação. 

In [74]:
# Cria um dicionário para os valores de 'Faixa Salarial' de forma que possa ser ordenada
nomes_faixas_salariais = {
    'Menos de R$ 1.000/mês' : '(A) Menos de R$ 1.000/mês',
    'de R$ 1.001/mês a R$ 2.000/mês' : '(B) R$ (1.001-2.000)/mês',
    'de R$ 2.001/mês a R$ 3000/mês' : '(C) R$ (2.001-3.000)/mês',
    'de R$ 3.001/mês a R$ 4.000/mês' : '(D) R$ (3.001-4.000)/mês',
    'de R$ 4.001/mês a R$ 6.000/mês' : '(E) R$ (4.001-6.000)/mês', 
    'de R$ 6.001/mês a R$ 8.000/mês' : '(F) R$ (6.001-8.000)/mês',
    'de R$ 8.001/mês a R$ 12.000/mês' : '(G) R$ (8.001-12.000)/mês',
    'de R$ 12.001/mês a R$ 16.000/mês' : '(H) R$ (12.001-16.000)/mês',
    'de R$ 16.001/mês a R$ 20.000/mês' : '(I) R$ (16.001-20.000)/mês',
    'de R$ 20.001/mês a R$ 25.000/mês' : '(J) R$ (20.001-25.000)/mês',
    'de R$ 25.001/mês a R$ 30.000/mês' : '(K) R$ (25.001-30.000)/mês',
    'de R$ 30.001/mês a R$ 40.000/mês' : '(L) R$ (30.001-40.000)/mês',
    'Acima de R$ 40.001/mês' : '(M) Acima de R$ 40.001/mês'
}

In [75]:
# Atribui os novos valores de 'Faixa salarial'
valores_faixas = np.array(list(nomes_faixas_salariais.values()))

In [76]:
# Substitui os valores de 'Faixa salarial' pelos definidos anteriormente
df_final['Faixa salarial'] = df_final['Faixa salarial'].replace(nomes_faixas_salariais)

In [77]:
# Verifica se os valores de 'Faixa salarial' foram alterados corretamente
assert (valores_faixas == np.sort(df_final['Faixa salarial'].unique())).all()

O valor de "Arquiteto de dados" em "Cargo Atual" é substituido por "Arquiteto de Dados".

In [78]:
# Cria filtro para cargos de arquiteto de dados
is_arquiteto_dados = (df_final['Cargo Atual'] == 'Arquiteto de dados') | (df_final['Cargo Atual'] == 'Arquiteto de Dados')

In [79]:
# Altera (ou mantém) valores dos cargos selecionados para 'Arquiteto de Dados'
df_final.loc[is_arquiteto_dados, 'Cargo Atual'] = 'Arquiteto de Dados'

In [80]:
# Atribui valores de 'Cargo Atual'
cargos_atuais = df_final['Cargo Atual'].unique()

In [81]:
# Verifica se o valor 'Analista de Negócios/Analista de BI/Business Analyst/Analytics Engineer' foi adicionado
#assert 'Analista de Negócios/Analista de BI/Business Analyst/Analytics Engineer' in cargos_atuais

In [82]:
# Verifica se o valor 'Analista de BI/BI Analyst/Analytics Engineer' foi removido
#assert 'Analista de BI/BI Analyst/Analytics Engineer' not in cargos_atuais

In [83]:
# Verifica se o valor 'Analista de Negócios/Business Analyst' foi removido
#assert 'Analista de Negócios/Business Analyst' not in cargos_atuais

In [84]:
# Verifica se o valor 'Arquiteto de Dados' não foi removido
assert 'Arquiteto de Dados' in cargos_atuais

In [85]:
# Verifica se o valor 'Arquiteto de dados' foi removido
assert 'Arquiteto de dados' not in cargos_atuais

Uma coluna "Conhece o Data Hackers" é criada e indica se o indivíduo conhece ou não o Data Hackers, já que existe uma coluna que indica quais iniciativa ele conhece.

In [86]:
# Cria filtro para identificar quem não conhecia o Data Hackers
coluna_iniciativas_data_hackers = 'Quais das iniciativas do Data Hackers que você já acessou/acompanhou?'
nao_conhece_data_hackers = (df_final[coluna_iniciativas_data_hackers] == 'Ainda não conhecia o Data Hackers')

In [87]:
# Cria coluna 'Conhece o Data Hackers' e atribui valores correspondentes
df_final['Conhece o Data Hackers'] = 'Sim'
df_final.loc[nao_conhece_data_hackers, 'Conhece o Data Hackers'] = 'Não'

In [88]:
contar_valores(df_final['Conhece o Data Hackers'])

,Quantidade,Porcentagem (%)
Sim,1738,75.21
Não,573,24.79


Os espaços em branco ao início e fim de valores de texto (object/string) são removidos.

In [89]:
# Definição da função remover_espacos_brancos_inicio_fim()
def remover_espacos_brancos_inicio_fim(df):
    """
    Retorna um dataframe cujos valores string não começam ou terminam com ' '.
    
    Argumentos:
    df -- DataFrame
    """
    formatar_string = lambda x: x.strip()
    return df.applymap(formatar_string)

In [90]:
# Remove espaços em branco do início e fim de todos os valores string do dataframe
df_final[colunas_string] = remover_espacos_brancos_inicio_fim(df_final[colunas_string])

<a id='save_csv'></a>
### 3.3.6. Salvar dataframe em um arquivo .csv

In [91]:
# Definição da função salvar_df_csv()
def salvar_df_csv(df, arquivo):
    """
    Salva o dataframe em um arquivo csv.
    
    Argumentos:
    df -- DataFrame
    arquivo -- string
    """
    df.to_csv(arquivo + '.csv', index=False)

In [92]:
# Salva os dados limpos em um arquivo
salvar_df_csv(df_final, arquivo='dados_state_of_data_2021')

---
<a id='eda'></a>
## 4. Análise dos dados

A etapa de análise dos dados contém as etapas de análise exploratória dos dados e visualização dos dados. Esta etapa extrai informações relevantes do conjunto de dados e gera gráficos que permitem obter e confirmar insights. Aqui são feitas consultas, filtragens, agrupamentos, contagens e plots gráficos.

In [93]:
# Função para agrupamentos por colunas e descrição do resultado
def contar_agrupamento(df, colunas_agrupamento, coluna_contagem='id'):
    """
    Retorna um dataframe de contagem por grupo. 
    
    Argumentos:
    df -- DataFrame 
    colunas_agrupamento -- list
    coluna_contagem -- string
    """
    df_agrupado = df.groupby(colunas_agrupamento, as_index=False)
    df_contagem = df_agrupado[coluna_contagem].count()
    df_contagem = df_contagem.rename(columns={coluna_contagem: 'Quantidade'})
    df_contagem = df_contagem.sort_values(by='Quantidade', ascending=False)
    
    df_contagem['Porcentagem'] = df_contagem['Quantidade']/df_contagem['Quantidade'].sum()
    df_contagem['Porcentagem'] = (df_contagem['Porcentagem'] * 100).round(2) 
    df_contagem['Porcentagem'] = df_contagem['Porcentagem'].astype(str) + '%'
    
    return df_contagem

---
<a id='pesquisa_geral'></a>
### 4.1. Pontos gerais da pesquisa

A primeira parte da análise é feita considerando as respostas de todos os participantes, em busca de consensos ou maiorias.

In [94]:
# Definição da função encontrar_respostas_mais_frequentes()
def encontrar_respostas_mais_frequentes(df, valor_desconsiderado = None):
    """
    Retorna um dataframe que mostra o(s) valor(es) de maior frequência e suas frequências absolutas e relativas. 
    
    Argumentos:
    df -- DataFrame 
    valor_desconsiderado -- string (Opcional - Valor padrão é None)
    """
    colunas_info = ['Pergunta', 'Quantidade de Respostas Distintas', 'Resposta Mais Frequente', 
                    'Maior Frequência Absoluta', 'Maior Frequência Relativa (%)']
    
    # Cria dataframe que armazenará as informações desejadas
    df_respostas = pd.DataFrame(columns = colunas_info)

    # Considera apenas colunas que não são numéricas
    colunas_qualitativas = df.select_dtypes(include=['object']).columns.values

    for coluna in colunas_qualitativas:
        
        serie = df[coluna]

        # Desconsidera colunas sem valores repetidos
        if serie.describe().freq == 1:
            continue
        
        # Desconsidera o valor determinado
        if valor_desconsiderado:
            serie = serie[serie != valor_desconsiderado]          
            
        # Cria um dataframe com as quantidades absolutas e relativas de cada valor
        contagem = serie.value_counts()
        percentagem = serie.value_counts(normalize=True)
        info_quali = pd.concat([contagem, percentagem], axis=1)
        info_quali = info_quali.reset_index(level=0)
        info_quali.columns = ['Resposta', 'Frequência Absoluta', 'Frequência Relativa']
        
        # Filtra valores de acordo com as frequências máximas
        eh_frequencia_maxima = info_quali['Frequência Absoluta'] == info_quali['Frequência Absoluta'].max()
        nao_eh_frequencia_unitaria = info_quali['Frequência Absoluta'] > 1
        info_quali = info_quali.loc[(eh_frequencia_maxima) & (nao_eh_frequencia_unitaria), :]
        
        # Renomeia as colunas outra vez
        map_nome_colunas = {'Resposta':'Resposta Mais Frequente', 'Frequência Absoluta': 'Maior Frequência Absoluta', 
                            'Frequência Relativa': 'Maior Frequência Relativa (%)'}
        info_quali.rename(columns=map_nome_colunas, inplace=True)
        
        # Converte a coluna para 
        info_quali['Maior Frequência Relativa (%)'] = (info_quali['Maior Frequência Relativa (%)'] * 100).round(2)
        
        # A pergunta será inserida conforme as respectivas respostas com frequências máximas iguais
        info_quali['Pergunta'] = coluna
        
        # Conta a quantidade de valores distintos da coluna
        info_quali['Quantidade de Respostas Distintas'] = serie.nunique()

        # Adiciona os valores filtrados e processados da coluna
        df_respostas = pd.concat([df_respostas, info_quali[colunas_info]], axis=0) 

    df_respostas.reset_index(drop=True, inplace=True)
    
    return df_respostas

In [95]:
# Encontra as repostas mais frequentes para cada coluna que não sejam 'Não Informado'
repostas_gerais = encontrar_respostas_mais_frequentes(df_final)
repostas_gerais = repostas_gerais[repostas_gerais['Resposta Mais Frequente'] != 'Não Informado']
repostas_gerais

,Pergunta,Quantidade de Respostas Distintas,Resposta Mais Frequente,Maior Frequência Absoluta,Maior Frequência Relativa (%)
0,Você está satisfeito na sua empresa atual?,3,Sim,1711,74.04
1,Faixa idade,8,25-29,752,32.54
2,Genero,3,Masculino,1877,81.22
3,Estado onde mora,21,São Paulo (SP),975,42.19
4,uf onde mora,21,SP,975,42.19
5,Regiao onde mora,5,Sudeste,1504,65.08
6,Nivel de Ensino,7,Graduação/Bacharelado,815,35.27
7,Área de Formação,10,Computação / Engenharia de Software / Sistemas de Informação/ TI,982,42.49
8,Qual sua situação atual de trabalho?,8,Empregado (CLT),1712,74.08
9,Setor,23,Tecnologia/Fábrica de Software,630,27.26


Analisando as respostas mais frequentes dos dados válidos:

- A principal localização é a região sudeste do Brasil, principalmente no estado de São Paulo. Uma parcela muito pequena dos entrevistados trabalham/moram no exterior.


- As características mais comuns, analisadas separadamente, são faixa etária de 25-29 anos; nível Pleno; experiência de 2 a 3 anos na área de dados; salários de 8.001 12.000 reais por mês; nivel de ensino de graduação/bacharelado em algum curso ligado a Computação/Informática/Engenharia.


- A linguagem mais utilizada é a SQL, sendo que SQL e Python é a combinação mais frequente. Apesar da existência de estruturas robustas que suportam consultas de dados não estruturados ou semiestruturados, a consulta de dados struturados ainda permanece e continuará essencial para diversos fins.


- A maioria (75.21 %) já conhecia o Data Hackers.


- Mais de 80 % dos profissionais são do gênero masculino. Grandes diferenças de proporções entre os gêneros costumam se apresentar em determinadas áreas de engenharias, exatas e principalmente de computação no Brasil até agora, mas há uma tendência de que essa diferença vá diminuindo devido a incetivos da comunidade e divulgação dos atrativos destas áreas para públicos mais amplos.  


- Mais de 70 % estão empregados em regime CLT e satisfeitos com a empresa atual, mostrando que as áreas analisadas, principalmente a de dados, são muito vantajosas e passam a ser mais desejadas por profissionais de outras áreas. 


- Mais da metade se encontra em um modelo de trabalho 100% remoto, considerando que a melhor forma de trabalhar é remotamente ou de modelo híbrido flexível. A pandemia do coronavírus tornou necessário a adoção deste modelo de trabalho a fim de possibilitar o isolamento social. De modo geral, os empregados também relatam em redes sociais, principalmente no Linkedin, os benefícios que este modelo de trabalho os proporcionaram, tais como melhor aproveitamento de tempo, já que não precisam enfrentar trânsito; economia de recursos financeiros, já que não precisaram se mudar ou se manter nas cidades onde as empresas se localizam; melhor qualidade de vida, por considerarem suas casas um ambiente melhor do que a empresa. Tanto 42.58 % disseram que procurariam outra oportunidade no modelo híbrido ou remoto caso a empresa decida pelo modelo 100% presencial. Em 2022, com o avanço considerável da vacinação e diminuição da crise sanitária da pandemia, não há um concenso entre chefes, líderes, CEOs, CTOs e afins se os empregados vão voltar a trabalhar apenas presencialmente ou não.



In [96]:
# Cria uma contagem por 'Atuacao'
df_atuacao_quantidade = contar_agrupamento(df_final, ['Atuacao'], coluna_contagem='id')
df_atuacao_quantidade = df_atuacao_quantidade.sort_values(by=['Atuacao'])

In [97]:
def salvar_fig(fig, titulo):
    
    # Remova acentos usando a biblioteca unidecode
    titulo_sem_acentos = unidecode.unidecode(titulo)
    
    # Substitua espaços por underscores
    titulo_formatado = re.sub(r'\s', '_', titulo_sem_acentos)
    
    # Remova outros caracteres especiais (exceto underscores)
    titulo_formatado = re.sub(r'[^A-Za-z0-9_]', '', titulo_formatado)
    
    # Obtenha o diretório atual de trabalho
    diretorio_atual = os.getcwd()
    
    # Crie o subdiretório 'imgs' dentro do diretório atual
    subdiretorio_imgs = os.path.join(diretorio_atual, 'imgs')
    
    # Verifique se a pasta 'imgs' existe e, se não, crie-a
    if not os.path.exists(subdiretorio_imgs):
        os.makedirs(subdiretorio_imgs)
    
    # Caminho completo para a imagem a ser salva
    caminho_imagem = os.path.join(subdiretorio_imgs, titulo_formatado + '.png')
    
    # Salve a imagem no caminho especificado
    pio.write_image(fig, caminho_imagem)

In [98]:
# Definição da função plotar_barra()
def plotar_barra(df, eixo_x, eixo_y, eixo_cores, cores, dados_extra=['Porcentagem'], titulo='' ):
    """
    Plota um gráfico de barras. 
    
    Argumentos:
    df -- DataFrame 
    eixo_x -- string 
    eixo_y  -- string (Opcional - Valor padrão é 'Quantidade')
    dados_extra --list (Opcional - Valor padrão é ['Porcentagem'])
    titulo -- string (Opcional - Valor padrão é '')
    """
    fig = px.bar(df, x=eixo_x, y=eixo_y,
                 hover_data=dados_extra,
                 title=titulo,
                 color=eixo_cores,
                 color_discrete_sequence=cores,
                 width=950, 
                 height=600)
    fig.update_layout(template=TEMA_BARRAS , title_text=titulo, title_x=0.5, showlegend=False)
    #pio.write_image(fig, '/imgs/' + titulo + '.png')
    salvar_fig(fig, titulo)

In [99]:
# Plota gráfico de barras
cores_barra_1 = ["orange", "orange", "orange", "purple", "purple"]
titulo_barra_1 = 'Quantidade por Atuação'
plotar_barra(df_atuacao_quantidade, eixo_x='Atuacao', eixo_y='Quantidade', eixo_cores='Atuacao', cores=cores_barra_1, titulo=titulo_barra_1)

A área de dados pode ser dividida em engenharia de dados, análise de dados e ciência de dados. Na pesquisa, o número de profissionais de análise de dados é aproximadamente o dobro dos de engenharia e ciência de dados. Esta tendência na área deve diminuir, já que, em 2021/2022 a procura por engenheiros de dados cresceu bastante conforme observou-se a necessidade de implementação de mais estruturas que extraiam e armazenem grandes quantidades de dados não estruturados e semi estruturados.

Um dos fluxos de trabalho mais comuns envolvendo essas áreas de atuação segue a ordem de engenharia de dados, depois análise de dados e então ciência de dados, sem considerar as demais áreas de uma empresa.

![Imagem de FluxoDoProjeto](Diagrama-Dados.png "Diagrama-Dados img")

In [100]:
# Definição da função plotar_box()
def plotar_box(df, eixo_x, eixo_y, eixo_cores, cores, titulo=''):
    """
    Plota um gráfico de caixa. 
    
    Argumentos:
    df -- DataFrame 
    eixo_x -- string 
    eixo_y  -- string 
    eixo_cores -- string 
    cores -- string 
    titulo -- string (Opcional - Valor padrão é '')
    """
    fig = px.box(df, x=eixo_x, y=eixo_y,
                 color=eixo_cores,
                 color_discrete_sequence=cores)
    fig.update_layout(template=TEMA_BOX, title_text=titulo, title_x=0.5)
    # fig.show()
    py.iplot(fig)
    #pio.write_image(fig, '/imgs/' + titulo + '.png')
    salvar_fig(fig, titulo)

In [101]:
# Plota gráfico de caixa (box plot) da Idade por Atuação
df_box = df_final[['Atuacao', 'Idade']].sort_values(by=['Atuacao'])
cores_box = ["orange", "orange", "orange", "purple", "purple"]
plotar_box(df_box, eixo_x='Atuacao', eixo_y="Idade", eixo_cores='Atuacao', cores=cores_box, titulo='Distribuição de Idades por Atuação')

As distribuições das idades dos profissionais que atuam em engenharia de dados e análise de dados são praticamente iguais, exceto por alguns outliers. Em ambas, as idades mínimas e máximas são, respectivamente, 18 e 47 anos. Os quartis são 25, 29 e 34 anos.

A distribuição das idades dos que atuam em ciência de dados é mais compacta se comparada as duas anteriores. Nesta distribuição, a idade mínima é de 20 anos, a máxima de 45 anos, e os quartis são 25, 28 e 33 anos.  

A faixa de idades de profissionais de gestão, como esperado, indica valores de idades maiores de mínimo, máximo e quartis se comparados aos de engenharia de dados, análise de dados e ciência de dados. Na área de gestão, muitas vezes é necessário ter mais experiência do que em outras áreas.

A distribuição de idade da atuação "Outra" é muito genérica e não serve para caracterizar um grupo muito específico, mas seus valores se assemelham com a aparente soma das outras distribuições.

In [102]:
# Definição de filtros por área de atuação
is_analise_dados = (df_final['Atuacao'] == 'Análise de Dados')
is_gestor = (df_final['Atuacao'] == 'Gestor')
is_ciencia_dados = (df_final['Atuacao'] == 'Ciência de Dados')
is_engenharia_dados = (df_final['Atuacao'] == 'Engenharia de Dados')
is_outra_atuacao = (df_final['Atuacao'] == 'Outra')

In [103]:
# Definição de filtros por cargos
is_analista_dados = (df_final['Cargo Atual'] == 'Analista de Dados/Data Analyst')
is_analista_bi = (df_final['Cargo Atual'] == 'Analista de BI/BI Analyst/Analytics Engineer')
is_engenheiro_dados = (df_final['Cargo Atual'] == 'Engenheiro de Dados/Data Engineer')
is_cientista_dados = (df_final['Cargo Atual'] == 'Cientista de Dados/Data Scientist')
is_cargo_analisado = (is_analista_dados) | (is_analista_bi) | (is_engenheiro_dados) | (is_cientista_dados)

In [104]:
# Definição de filtros por estar dentro ou fora do Brasil
is_in_Brasil = (df_final['Regiao onde mora'] != 'Exterior')
not_is_in_Brasil = (df_final['Regiao onde mora'] == 'Exterior')

In [105]:
# Cria uma contagem por 'Cargo Atual'
df_cargo_quantidade = contar_agrupamento(df_final, ['Cargo Atual'], coluna_contagem='id')

In [106]:
# Plota gráfico de barra de 'Quantidade' por 'Cargo Atual'
# cores_barra_2 = ["orange", "orange", "orange", "purple", "purple"]
cores_barra_2 = ["purple"] * df_cargo_quantidade.shape[0]
cores_barra_2 = ["orange"] * 4 + cores_barra_2[4:]
titulo_barra_2 = 'Quantidade por Cargo Atual'
plotar_barra(df_cargo_quantidade, eixo_x='Quantidade', eixo_y='Cargo Atual', eixo_cores='Cargo Atual', cores=cores_barra_2, titulo=titulo_barra_2)

In [107]:
# Definição da função tabela_cruzamento()
def tabela_cruzamento(df, x, y, normalizar=False):
    """
    Retorna uma tabela de contagem do cruzamento de duas colunas. 
    
    Argumentos:
    df -- DataFrame 
    x -- string 
    y  -- string 
    normalizar -- bool (Opcional - Valor padrão é False)
    """
    return pd.crosstab(df[y], df[x], normalize=normalizar)

In [108]:
# Exibe a tabela de cruzamento de 'Cargo Atual' com 'Atuacao'
tabela_cruzamento(df_final, x='Atuacao', y='Cargo Atual', normalizar=False)

Atuacao,Análise de Dados,Ciência de Dados,Engenharia de Dados,Gestor,Outra
Cargo Atual,,,,,
Analista Administrativo,8,2,0,0,7
Analista de BI/BI Analyst/Analytics Engineer,277,6,45,0,3
Analista de Dados/Data Analyst,276,17,17,0,8
Analista de Inteligência de Mercado/Market Intelligence,14,2,0,0,2
Analista de Marketing,10,0,0,0,4
Analista de Negócios/Business Analyst,75,3,4,0,11
Analista de Sistemas/Analista de TI,4,1,0,0,8
Arquiteto de Dados,1,0,8,0,0
Cientista de Dados/Data Scientist,47,290,16,0,0


Os quatro cargos mais frequentes na pesquisa são "Analista de Dados/Data Analyst", "Analista de BI/BI Analyst/AnalyticsEngineer", "Engenheiro de Dados/Data Engineer" e "Cientista de Dados/Data Scientist" e representam cerca de 56,2 % dos valores escolhidos para esta pergunta. Mais adiante, cada um destes cargos será analisado seperadamente e ao final haverá um predição de salários para perfis destes cargos analisando algumas informações conjuntas. 

In [111]:
# Realiza a contagem de profissionais por Faixa Salarial
df_cargo_salario_quantidade = contar_agrupamento(df_final[is_cargo_analisado], ['Faixa salarial'], coluna_contagem='id')
df_cargo_salario_quantidade = df_cargo_salario_quantidade.sort_values(by=['Faixa salarial'])

In [112]:
# Definição da função plotar_area()
def plotar_area(df, eixo_x, eixo_y='Quantidade', dados_extra=['Porcentagem'], titulo='', categorias=None):
    """
    Plota um gráfico de área. 
    
    Argumentos:
    df -- DataFrame 
    eixo_x -- string 
    eixo_y  -- string (Opcional - Valor padrão é 'Quantidade')
    dados_extra --list (Opcional - Valor padrão é ['Porcentagem'])
    titulo -- string (Opcional - Valor padrão é '')
    categorias -- string (Opcional - Valor padrão é None)
    """
    fig = px.area(df, x=eixo_x, y=eixo_y,
              color=categorias,
              hover_data=dados_extra)
    fig.update_layout(title_text=titulo, title_x=0.5,
                     legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99))
    #fig.show()
    py.iplot(fig)
    #pio.write_image(fig, '/imgs/' + titulo + '.png')
    salvar_fig(fig, titulo)

In [113]:
# Plota área da quantidade por faixa salarial
plotar_area(df_cargo_salario_quantidade, eixo_x='Faixa salarial', eixo_y='Quantidade', titulo = 'Quantidade por faixa salarial', categorias=None)

Uma parte considerável dos respondentes possui salaário entre 1.000 e 3.000 reais por mês, ou entre 12.000 e 16.000 reais por mês. A maior parte dos salários se encontra entre 4.000 e 12.000 reais por mês, mostrando que as áreas analisadas, principalmente a de dados, são muito rentáveis mesmo no Brasil. Cerca de 1% ganha menos de 1.000 reais por mês. Dentre os entrevistados. Há profissionais que recebem mais 40.000 reais por mês.


In [114]:
# Definição da função plotar_linha()
def plotar_linha(df, eixo_x, eixo_y='Quantidade', dados_extra=['Porcentagem'], titulo='Titulo', categorias=None):
    """
    Plota um gráfico de linhas. 
    
    Argumentos:
    df -- DataFrame 
    eixo_x -- string 
    eixo_y  -- string (Opcional - Valor padrão é 'Quantidade')
    dados_extra --list (Opcional - Valor padrão é ['Porcentagem'])
    titulo -- string (Opcional - Valor padrão é 'Titulo')
    categorias -- string (Opcional - Valor padrão é None)
    """
    fig = px.line(df, x=eixo_x, y=eixo_y,
                  hover_data=dados_extra,
                  color=categorias
                  )
    fig.update_layout(title_text=titulo, title_x=0.5,
                    legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99)
    )
    #fig.show()
    py.iplot(fig)
    #pio.write_image(fig, '/imgs/' + titulo + '.png')
    salvar_fig(fig, titulo)

In [115]:
# Realiza a contagem de profissionais por 'Cargo Atual' e 'Faixa salarial'
df_cargo_salario_quantidade_2 = contar_agrupamento(df_final[is_cargo_analisado], ['Cargo Atual', 'Faixa salarial'], coluna_contagem='id')
df_cargo_salario_quantidade_2 = df_cargo_salario_quantidade_2.sort_values(by=['Faixa salarial','Cargo Atual'])

In [116]:
# Plota linhas de quantidade por 'Faixa salarial' por 'Cargo Atual'
plotar_linha(df_cargo_salario_quantidade_2, eixo_x='Faixa salarial', eixo_y='Quantidade', titulo='Quantidade por faixa salarial e cargo',categorias='Cargo Atual')

A faixa de salário mais frequente de salário para "Analista de BI/BI Analyst/AnalyticsEngineer" foi a entre 4.001 a 6.000 reais por mês, seguida pelas faixas entre 2.001 a 3.000 e entre 6.001 a 12.000 reais por mês.
 
 
A distribuição das faixas salariais para "Analista de Dados/Data Analyst" é a que mais se parece com a do conjunto total dos dados. A faixa de salário mais frequente para este cargo também foi entre 4.001 a 6.000, assim como para o cargo anterior. A segunda faixa mais frequente foi entre 6.001 a 12.000 reais por mês.
 
 
A faixa de salário mais frequente de salário para "Cientista de Dados/Data Scientist" foi a entre 8.001 a 12.000. A quantidade de profissionais que ganha entre mais 3.000 reais por mês cresce, quase que linearmente, até chegar a faixa até 8.000, e após isso cai, também quase linearmente, até chegar a faixa de 20.001 a 25.000 reais por mês.


A distribuição das faixas salariais para "Engenheiro de Dados/Data Engineer" é que mais se assemelha com uma função sino, a faixa salarial mais frequente é a entre 6.001 a 8.000 reais por mês. A maioria daqueles que possuem salário acima de 30.000 reais por mês são "Engenheiro de Dados/Data Engineer", e o restante são "Cientista de Dados/Data Scientist". 
 
 
Todas as distribuições possuem apenas uma moda.

---
<a id='engenheiro_dados'></a>
### 4.2. Engenheiro de Dados

In [117]:
# Define quais perguntas serão avaliadas para os engenheiros de dados
perguntas_engenheiro_dados = [
    'id',
    'Atuacao',
    'Cargo Atual',
    'Idade',
    'Nivel',
    'Nivel de Ensino',
    'Área de Formação',
    'Qual sua situação atual de trabalho?',
    'Faixa salarial',
    'Quanto tempo de experiência na área de dados você tem?',
    'Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?',
    'Mesmo que esse não seja seu cargo formal, você considera que sua atuação no dia a dia, reflete alguma das opções listadas abaixo?',
    'Quais das fontes de dados listadas você já analisou ou processou no trabalho?',
    'Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?',
    'Quais das linguagens listadas abaixo você utiliza no trabalho?',
    'Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?',
    'Quais dos bancos de dados/fontes de dados listados abaixo você utiliza no trabalho?',
    'Quais das opções de Cloud listadas abaixo você utiliza no trabalho?',
    'Quais das opções abaixo fazem parte da sua rotina no trabalho atual como engenheiro de dados?',
    'Quais as ferramentas/tecnologias de ETL que você utiliza no trabalho como Data Engineer?',
    'Qual tecnologia utilizada como plataforma do Data Lake?',
    'Qual tecnologia utilizada como plataforma do Data Warehouse?',
    'Em qual das opções abaixo você gasta a maior parte do seu tempo de trabalho?'
]

In [118]:
# Filtra os profissionais de Engenharia de dados que trabalham no Brasil
df_engenheiro_dados = df_final.loc[is_engenharia_dados & is_engenheiro_dados & is_in_Brasil, perguntas_engenheiro_dados]

In [119]:
# Exibe as respostas mais frequentes paras perguntas definidas para os engenheiros de dados no Brasil
encontrar_respostas_mais_frequentes(df_engenheiro_dados, valor_desconsiderado='Não Informado')

,Pergunta,Quantidade de Respostas Distintas,Resposta Mais Frequente,Maior Frequência Absoluta,Maior Frequência Relativa (%)
0,Atuacao,1,Engenharia de Dados,276,100.00
1,Cargo Atual,1,Engenheiro de Dados/Data Engineer,276,100.00
2,Nivel,3,Pleno,104,37.68
3,Nivel de Ensino,6,Graduação/Bacharelado,111,40.22
4,Área de Formação,9,Computação / Engenharia de Software / Sistemas de Informação/ TI,184,69.17
5,Qual sua situação atual de trabalho?,6,Empregado (CLT),214,77.54
6,Faixa salarial,13,(F) R$ (6.001-8.000)/mês,61,22.10
7,Quanto tempo de experiência na área de dados você tem?,6,de 2 a 3 anos,77,27.90
8,Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?,7,Não tive experiência na área de TI/Engenharia de Software antes de começar a trabalhar na área de dados,61,22.10
9,"Mesmo que esse não seja seu cargo formal, você considera que sua atuação no dia a dia, reflete alguma das opções listadas abaixo?",1,"*Engenharia de Dados:* Modela soluções de arquitetura de dados; define modelagens de repositórios de dados (Data Lake, Data Warehouse, Data Lakehouse); desenvolve estratégias de aquisição de dados, recuperação de informação e pipelines de dados.",276,100.00


In [120]:
# Definição da função contar_valores_dummies()
def contar_valores_dummies(df, coluna, separador=',', is_forma_crescente=False, valor_outro='Outros', perc_outro = 10.00):
    """
    Retorna um dataframe com os valores dummies de uma coluna e sua contagem absoluta e percentual. 
    
    Argumentos:
    df -- DataFrame 
    coluna -- string 
    separador  -- string (Opcional - Valor padrão é ',')
    is_forma_crescente -- bool (Opcional - Valor padrão é False)
    valor_outro  -- string (Opcional - Valor padrão é 'Outros')
    perc_outro  -- int (Opcional - Valor padrão é '10.00')
    """
    serie = df[coluna].str.get_dummies(separador).sum()

    serie = serie.reset_index()
    serie.columns = ['Valor', 'Quantidade']
    quantidades, valores = [], []
    for q in serie['Quantidade'].unique():
        valor = serie.loc[serie['Quantidade'] == q, 'Valor']
        quantidades.append(q)
        valores.append(', '.join(valor.unique()))
    
    df_cont = pd.DataFrame({'Valor' : valores, 'Quantidade' : quantidades})
    df_cont['Porcentagem (%)'] = (df_cont['Quantidade']/(df_cont['Quantidade'].sum()) * 100).round(2)
    
    df_outros = df_cont[(df_cont['Porcentagem (%)'] < perc_outro)]
    pct_outros = ((df_outros['Quantidade']/(df_cont['Quantidade'].sum()) * 100).round(2).sum())
    map_outros = {'Valor' : [valor_outro], 'Quantidade' : df_outros['Quantidade'].sum(), 'Porcentagem (%)' : pct_outros}
    df_outros_juntos = pd.DataFrame(map_outros)

    df_cont = df_cont[df_cont['Porcentagem (%)'] > perc_outro]
    df_cont = pd.concat([df_cont, df_outros_juntos])
    
    df_cont = df_cont.reset_index(drop=True)
    df_cont = df_cont.sort_values(by=['Quantidade'], ascending=is_forma_crescente)
    
    return df_cont

In [121]:
# Realiza a contagem de tarefas de rotina de Engenheiros de Dados
pergunta_hab_eng = 'Quais das opções abaixo fazem parte da sua rotina no trabalho atual como engenheiro de dados?'
contar_valores_dummies(df_engenheiro_dados, pergunta_hab_eng, separador=', ', is_forma_crescente= False, perc_outro=2.0)

,Valor,Quantidade,Porcentagem (%)
5,"Desenvolvo pipelines de dados utilizando linguagens de programação como Python, Java etc., Scala",236,19.98
7,"Modelo soluções de arquitetura de dados, criando componentes de ingestão de dados, transformação e recuperação da informação.",186,15.75
2,Crio consultas através da linguagem SQL para exportar informações e compartilhar com as áreas de negócio.,179,15.16
1,"Atuo na modelagem dos dados, Data Marts etc., com o objetivo de criar conjuntos de dados como Data Warehouses",147,12.45
3,"Cuido da qualidade dos dados, metadados e dicionário de dados.",146,12.36
4,"Dataflow etc., Realizo construções de ETL's em ferramentas como Pentaho, Talend",119,10.08
6,Desenvolvo/cuido da manutenção de repositórios de dados baseados em streaming de eventos como Data Lakes e Data Lakehouses.,114,9.65
0,"Atuo na integração de diferentes fontes de dados através de plataformas proprietárias como Stitch Data, Fivetran etc.",53,4.49
8,Outros,1,0.08


A área de Engenharia de Dados é, de forma geral, caracterizada pelas responsabilidades de: 
- Modelar soluções de arquitetura de dados; 
- Definir modelagens de repositórios de dados (Data Lake, Data Warehouse, Data Lakehouse); 
- Desenvolver estratégias de aquisição de dados, recuperação de informação e pipelines de dados;


As tarefas mais comuns da rotina de um engenheiro de dados, daquelas com maiores frequências para a de menores frequências, são:
- Desenvolver pipelines de dados utilizando linguagens de programação como Python, Scala, Java etc.
- Modelar soluções de arquitetura de dados, criando componentes de ingestão de dados, transformação e recuperação da informação. 
- Criar consultas através da linguagem SQL para exportar informações e compartilhar com as áreas de negócio.
- Atuar na modelagem dos dados, com o objetivo de criar conjuntos de dados como Data Warehouses, Data Marts etc., 
- Cuidar da qualidade dos dados, metadados e dicionário de dados.	
- Realizar construções de ETL's em ferramentas como Pentaho, Talend, Dataflow etc.
- Desenvolver/cuidar da manutenção de repositórios de dados baseados em streaming de eventos como Data Lakes e Data Lakehouses. 
- Atuar na integração de diferentes fontes de dados através de plataformas proprietárias como Stitch Data, Fivetran etc.

As respostas mais frequentes dos profissionais de Engenharia de dados no Brasil em 2021:

- Graduação/Bacharelado em Computação/Engenharia de Software/Sistemas de Informação/TI.
- Não teve experiência na área de TI/Engenharia de Software antes de começar a trabalhar na área de dados.
- Experiência de 2 a 3 anos	na área de dados.
- São de nível pleno.
- Empregados em regime (CLT).
- Salário de (6.001-8.000) reais/mês.
- Dados relacionais (estruturados em bancos SQL), Dados armazenados em bancos NoSQL como fontes de dados mais utilizadas, além de usarem Textos/Documentos, Planilhas.
- Utilizam as linguagens SQL, Python, sendo Python a mais utilizada.
- SQL SERVER e Google BigQuery como bancos de dados/fontes de dados listados abaixo você utiliza no trabalho.
- Amazon Web Services (AWS) como principal opção de Cloud.
- Scripts Python, SQL & Stored Procedures, Apache Airflow como ferramentas/tecnologias de ETL.
- Amazon S3 + Redshift + Athena como principal  plataforma do Data Lake.
- AWS Redshift como principal plataforma do Data Warehouse.

In [122]:
# Definição da função plotar_treemap_niveis
def plotar_treemap_niveis(df, hierarquia, titulo, raiz='Raiz'):
    """
    Plota um gráfico de mapas de árvore. 
    
    Argumentos:
    df -- DataFrame (com colunas 'Valor', 'Quantidade' e 'Porcentagem')
    hierarquia -- list
    titulo -- string
    raiz -- string (Opcional - 'Raiz')
    """
    hierarquia = [px.Constant(raiz)] + hierarquia
    fig = px.treemap(df, path=hierarquia, values='Quantidade', hover_data=['Porcentagem'], color=hierarquia[-1])
    fig.update_layout(template=TEMA_TREEMAP, title_text=titulo, title_x=0.5)
    #fig.show()
    py.iplot(fig)
    #pio.write_image(fig, '/imgs/' + titulo + '.png')
    salvar_fig(fig, titulo)

In [123]:
# Plota um mapa de árvore de nível de ensino por nivel de carreira para engenheiros de dados
df_niveis_eng = contar_agrupamento(df_engenheiro_dados, colunas_agrupamento=['Nivel', 'Nivel de Ensino'], coluna_contagem='id')
hierarquia=['Nivel', 'Nivel de Ensino']
df_niveis_eng = df_niveis_eng.sort_values(by=hierarquia, ascending=False)
plotar_treemap_niveis(df_niveis_eng, hierarquia, titulo='Engenheiros de Dados por Nivel e Nivel de Ensino', raiz='Engenheiro de dados')

O níveis de ensino de Estudante de Graduação Graduação/Bacharelado, Pós-Graduação são os mais frequentes para os três níveis de carreira. Considerando apenas estes três niveis de ensino, parece haver uma correlação entre o nível de ensino e o nível de carreira, mas a distribuição dos outros níveis de ensino parece contrariar esta hipótese.

In [124]:
# Realiza a contagem de Engenheiros de Dados por Nivel e Faixa Salarial
df_salario_engenheiro = contar_agrupamento(df_engenheiro_dados, ['Nivel', 'Faixa salarial'], coluna_contagem='id')
df_salario_engenheiro = df_salario_engenheiro.sort_values(by=['Faixa salarial'])

In [125]:
# Removendo o outlier que prejudica a visualização do gráfico
nao_outlier_eng = (df_salario_engenheiro['Nivel'] != 'Junior') & (df_salario_engenheiro['Faixa salarial'] != '(J) R$ (20.001-25.000)/mês')
df_salario_engenheiro = df_salario_engenheiro[nao_outlier_eng]

In [ ]:
# Plota áreas da quantidade por faixa salarial por nivel
titulo_area_eng = 'Quantidade de Engenheiros de Dados por Faixa Salarial e Nivel'
plotar_area(df_salario_engenheiro, eixo_x='Faixa salarial', eixo_y='Quantidade', categorias='Nivel', titulo=titulo_area_eng)

Os engenheiros de dados de nivel júnior, em sua maioria, ganham até 6.000 reais por mês, e alguns poucos ganham entre 6.000 e 8.000 reais por mês. Os engenheiros de dados de nivel pleno, em sua maioria, ganham até 6.000 e 8.000 reais por mês, e alguns poucos ganham entre 8.000 e 16.000 reais por mês. Os engenheiros de dados de nivel sênior, parecem ter salário inicial de 6.000 reais por mês, e em sua maioria, ganham até 30.000 reais por mês, e alguns poucos ganham mais do que isso.

In [ ]:
# Define as perguntas cujas respostas são analisadas por gráficos de pizzas
perguntas_pizza_eng = [    
    'Quais das opções de Cloud listadas abaixo você utiliza no trabalho?',
    'Quais as ferramentas/tecnologias de ETL que você utiliza no trabalho como Data Engineer?',
    'Qual tecnologia utilizada como plataforma do Data Lake?',
    'Qual tecnologia utilizada como plataforma do Data Warehouse?'
]

In [ ]:
# Consulta quantidades das respostas para Engenheiros de Dados
df_perguntas_eng =[]
for p in perguntas_pizza_eng:
    df_perguntas_eng.append(contar_valores_dummies(df_engenheiro_dados, p, separador=', ', is_forma_crescente= False,perc_outro=5.0))

In [ ]:
# Formatação de valores para o gráfico de pizza
valor_replace_pizza_1 = {'Servidores On Premise/Não utilizamos Cloud' :'Não utiliza Cloud'}
df_perguntas_eng[0]['Valor'] = df_perguntas_eng[0]['Valor'].replace(valor_replace_pizza_1)

In [ ]:
# Definição da função plotar_pizzas()
def plotar_pizzas(dfs, titulo_principal, titulos):
    """
    Plota gráficos de pizzas lado a lado. 
    
    Argumentos:
    dfs -- list de DataFrames (com colunas 'Valor', 'Quantidade' e 'Porcentagem')
    titulo_principal -- string
    titulos -- list
    """
    n_colunas = len(dfs)
    tipos_gráficos = [{"type": "pie"} for j in range(n_colunas)]
    
    fig = make_subplots(rows=1, cols=n_colunas, specs=[tipos_gráficos])  
    for j in range(n_colunas):
        df = dfs[j]
        fig.add_trace(
        go.Pie(labels=df['Valor'], values=df['Quantidade'],  title='<b>' + titulos[j] +'</b>'
               ,titleposition='top center', showlegend=True,  
               legendgroup=('Grupo ' + str(j)), 
               legendgrouptitle_text=titulos[j],
               name=titulos[j]
              ),
        row=1, col=j+1
        )
    fig.update_layout(title_text=titulo_principal, title_font_size=20, title_x=0.5, template=TEMA_PIZZA)
    #fig.show()
    py.iplot(fig)
    #pio.write_image(fig, '/imgs/' + titulo_principal + '.png')
    salvar_fig(fig, titulo_principal)

In [ ]:
# Plote de 2 gráficos de pizza para ferramentas usadas por Engenheiros de Dados
titulo_pizza_eng_1 = 'Ferramentas usadas por Engenheiros de Dados'
subtitulos_pizza_eng_1 = ['Opções de Cloud', 'Ferramentas/tecnologias de ETL']
plotar_pizzas(dfs = df_perguntas_eng[:2], titulo_principal=titulo_pizza_eng_1, titulos=subtitulos_pizza_eng_1)

A Amazon Web Service (AWS) lidera com folga como a opção de cloud utilizada por engenheiros de dados, sendo escolhida por 41% dos respondentes, contra 25,3% do Google Cloud (GCP) e 19,8% do Azure (Microsoft).

A principal forma que os engenheiros de dados realizam ETL é através de scripts Python, sendo seguido por uso do SQL, mostrando que muito do trabalho de ETL ainda é feita com foco em linguagens e não necessariamente em ferramentas.

In [ ]:
# Formatação de valores para o gráfico de pizza
valor_replace_pizza_2 = {'Amazon S3 + Redshift + Athena' : 'AmazonS3+Redshift+Athena'}
df_perguntas_eng[2]['Valor'] = df_perguntas_eng[2]['Valor'].replace(valor_replace_pizza_2)

In [ ]:
# Plote de 2 gráficos de pizza para plataformas usadas por Engenheiros de Dados
titulo_pizza_eng_2 = 'Plataformas usadas por Engenheiros de Dados'
subtitulos_pizza_eng_2 = ['Plataforma do Data Lake', 'Plataforma do Data Warehouse']
plotar_pizzas(dfs = df_perguntas_eng[2:], titulo_principal=titulo_pizza_eng_2, titulos=subtitulos_pizza_eng_2)

Assim como foi para a escolha de opções de cloud, as tecnologias da Amazon, Google e Azure (Microsoft), respectivamente, apresentam as escolhas mais frequentes para plataformas de Data Lake e de Data Warehouse.

---
<a id='analista_bi'></a>
### 4.3. Analista de BI

In [ ]:
# Define quais perguntas serão avaliadas para os analistas de bi
perguntas_analise_bi = [
'id',
'Nivel de Ensino',
'Área de Formação',
'Qual sua situação atual de trabalho?',
'Cargo Atual',
'Nivel',
'Faixa salarial',
'Quanto tempo de experiência na área de dados você tem?',
'Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?',
'Quais são os 3 maiores desafios que você tem como gestor no atual momento?',
'Mesmo que esse não seja seu cargo formal, você considera que sua atuação no dia a dia, reflete alguma das opções listadas abaixo?',
'Quais das fontes de dados listadas você já analisou ou processou no trabalho?',
'Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?',
'Quais das linguagens listadas abaixo você utiliza no trabalho?',
'Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?',
'Quais dos bancos de dados/fontes de dados listados abaixo você utiliza no trabalho?',
'Quais das opções de Cloud listadas abaixo você utiliza no trabalho?',
'Quais as Ferramentas de Business Intelligence você utiliza no trabalho?',
'Quais as ferramentas de gestão de Qualidade de dados, Metadados e catálogo de dados você utiliza no trabalho?',
'Em qual das opções abaixo você gasta a maior parte do seu tempo?',
'Quais das opções abaixo fazem parte da sua rotina no trabalho atual com análise de dados?',
'Quais as ferramentas/tecnologias de ETL que você utiliza no trabalho como Data Analyst?',
'Em qual das opções abaixo você gasta a maior parte do seu tempo de trabalho?',
'Quais das opções abaixo fazem parte da sua rotina no trabalho atual com ciência de dados?',
'Quais as técnicas e métodos listados abaixo você costuma utilizar no trabalho?',
'Sua empresa utiliza alguma das ferramentas listadas para dar mais autonomia em análise de dados para as áreas de negócio?'
]

In [ ]:
# Filtra os profissionais de Engenharia de dados que trabalham no Brasil
df_analista_bi = df_final.loc[is_analise_dados & is_analista_bi & is_in_Brasil, perguntas_analise_bi]

In [ ]:
# Exibe as respostas mais frequentes paras perguntas definidas para os profissionais de Engenharia de dados no Brasil
encontrar_respostas_mais_frequentes(df_analista_bi, valor_desconsiderado='Não Informado')

,Pergunta,Quantidade de Respostas Distintas,Resposta Mais Frequente,Maior Frequência Absoluta,Maior Frequência Relativa (%)
0,Nivel de Ensino,6,Graduação/Bacharelado,104,37.68
1,Área de Formação,9,Computação / Engenharia de Software / Sistemas de Informação/ TI,106,39.11
2,Qual sua situação atual de trabalho?,5,Empregado (CLT),205,74.28
3,Cargo Atual,1,Analista de BI/BI Analyst/Analytics Engineer,276,100.00
4,Nivel,3,Júnior,107,38.77
5,Faixa salarial,10,(E) R$ (4.001-6.000)/mês,78,28.26
6,Quanto tempo de experiência na área de dados você tem?,7,de 2 a 3 anos,71,25.72
7,Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?,7,Não tive experiência na área de TI/Engenharia de Software antes de começar a trabalhar na área de dados,120,43.48
8,"Mesmo que esse não seja seu cargo formal, você considera que sua atuação no dia a dia, reflete alguma das opções listadas abaixo?",1,"*Análise de Dados/BI:* Extrai e cruza dados unindo diferentes fontes da informação; analisa dados visando identificar padrões, gerar insights e levantar perguntas; desenvolve dashboards, relatórios e visualizações de dados em ferramentas de BI.",276,100.00
9,Quais das fontes de dados listadas você já analisou ou processou no trabalho?,69,"Dados relacionais (estruturados em bancos SQL), Textos/Documentos, Planilhas",44,15.94


In [ ]:
pergunta_tempo_abi = 'Em qual das opções abaixo você gasta a maior parte do seu tempo de trabalho?'
contar_valores_dummies(df_analista_bi, pergunta_tempo_abi, separador=', ', is_forma_crescente= False, perc_outro=2.0)

,Valor,Quantidade,Porcentagem (%)
4,"Looker, Qlik etc., Realizando construções de dashboards em ferramentas de BI como PowerBI, Tableau",201,43.41
2,Criando consultas através da linguagem SQL para exportar informações e compartilhar com as áreas de negócio.,113,24.41
5,"Processando e analisando dados utilizando linguagens de programação como Python, R etc.",49,10.58
0,"Airflow, Dataflow etc., Desenvolvendo/cuidando da manutenção de ETL's utilizando tecnologias como Talend, Pentaho",32,6.91
3,Desenvolvendo/cuidando da manutenção de planilhas para atender as áreas de negócio.,26,5.62
6,Outros,22,4.75
1,"Atuando na modelagem dos dados, Data Marts etc., com o objetivo de criar conjuntos de dados como Data Warehouses",20,4.32


A área de Análise de Dados/BI é, de forma geral, caracterizada pelas responsabilidades de:
- Extrair e cruza dados unindo diferentes fontes da informação; 
- Analisar dados visando identificar padrões, gerar insights e levantar perguntas; 
- Desenvolver dashboards, relatórios e visualizações de dados em ferramentas de BI.
    
As duas principais tarefas da rotina dos analistas de BI são:
- Realizar construções de dashboards em ferramentas de BI como PowerBI, Tableau, Looker, Qlik etc.
- Criar consultas através da linguagem SQL para exportar informações e compartilhar com as áreas de negócio.

Além destas tarefas, outras que analistas de BI pode fazer são:
- Processar e analisar dados utilizando linguagens de programação como Python, R etc.
- Desenvolver/cuidar da manutenção de ETL's utilizando tecnologias como Talend, Pentaho, Airflow, Dataflow etc.
- Desenvolver/cuidar da manutenção de planilhas para atender as áreas de negócio.
- Atuando na modelagem dos dados, Data Marts etc., com o objetivo de criar conjuntos de dados como Data Warehouses.	


As respostas mais frequentes dos analistas de BI no Brasil em 2021:
- Graduação/Bacharelado em Computação/Engenharia de Software/Sistemas de Informação/TI.
- Não teve experiência na área de TI/Engenharia de Software antes de começar a trabalhar na área de dados.
- Experiência de 2 a 3 anos na área de dados.
- São de nível Júnior.
- Empregados em regime (CLT).
- Salário de (4.001-6.000) reais/mês.
- Dados relacionais (estruturados em bancos SQL), Planilhas como fontes de dados mais utilizadas, além de usarem Textos/Documentos.
- Utilizam as linguagens SQL e Python, sendo SQL a mais utilizada.
- Amazon Web Services (AWS) como principal opção de Cloud.

In [ ]:
# Plota um mapa de árvore de nível de ensino por nivel de carreira para Analistas de BI
df_niveis_abi = contar_agrupamento(df_analista_bi, colunas_agrupamento=['Nivel', 'Nivel de Ensino'], coluna_contagem='id')
hierarquia=['Nivel', 'Nivel de Ensino']
df_niveis_abi = df_niveis_abi.sort_values(by=hierarquia, ascending=False)
plotar_treemap_niveis(df_niveis_abi, hierarquia, titulo='Analistas de BI por Nivel e Nivel de Ensino', raiz='Analista de BI')

Para todos os três niveis de carreira, os três principais niveis de ensino são estudante de graduação, graduado/bacharel e pós-graduado. Para os níveis de júnior e pleno, o nivel de graduação é o mais frequente, enquanto que para o nivel sênior, a pós-graduação é o nivel de ensino mais frequente.  

In [ ]:
# Realiza a contagem de Analistas de BI por Nivel e Faixa Salarial
df_salario_analista_bi = contar_agrupamento(df_analista_bi, ['Nivel', 'Faixa salarial'], coluna_contagem='id')
df_salario_analista_bi = df_salario_analista_bi.sort_values(by=['Faixa salarial'])

In [ ]:
# Plota áreas da quantidade por faixa salarial por nivel
titulo_area_abi = 'Quantidade de Analistas de BI por Faixa Salarial e Nivel'
plotar_area(df_salario_analista_bi, eixo_x='Faixa salarial', eixo_y='Quantidade', categorias='Nivel', titulo=titulo_area_abi)

Os analistas de BI de nivel júnior, têm uma extensa faixa salarial  e em sua maioria, ganham até 6.000 reais por mês, e alguns poucos ganham entre 6.000 e 8.000 reais por mês. Os analistas de BI de nivel pleno, em sua maioria, ganham até 4.000 e 6.000 reais por mês, e alguns poucos ganham mais do que 6.000 reais por mês. Os analistas de BI de nivel sênior, parecem ter salário inicial de 4.000 reais por mês, e em sua maioria, ganham entre 8.001 e 12.000 reais por mês.

In [ ]:
# Define as perguntas cujas respostas são analisadas por gráficos de pizzas
perguntas_pizza_abi = [    
    'Quais as Ferramentas de Business Intelligence você utiliza no trabalho?',
    'Quais as ferramentas/tecnologias de ETL que você utiliza no trabalho como Data Analyst?',
    'Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?',
    'Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?'
]

In [ ]:
# Consulta quantidades das respostas para Analistas de Dados
df_perguntas_abi =[]
for p in perguntas_pizza_abi:
    df_perguntas_abi.append(contar_valores_dummies(df_analista_bi, p, separador=', ', is_forma_crescente= False,perc_outro=5.0))

In [ ]:
# Formatação de valores para o gráfico de pizza
valor_replace_pizza_3 = {'Fazemos todas as análises utilizando apenas Excel ou planilhas do google' : 'Excel/Planilhas'}
df_perguntas_abi[0]['Valor'] = df_perguntas_abi[0]['Valor'].replace(valor_replace_pizza_3)

In [ ]:
# Formatação de valores para o gráfico de pizza
valor_replace_pizza_4 = {'Não utilizo ferramentas de ETL, Pentaho' : 'Não utiliza', 'SQL Server Integration Services (SSIS)' : 'SSIS'}
df_perguntas_abi[1]['Valor'] = df_perguntas_abi[1]['Valor'].replace(valor_replace_pizza_4)

In [ ]:
# Plote de 2 gráficos de pizza para ferramentas usadas por Analistas de BI
titulo_pizza_abi_1 = 'Ferramentas usadas por Analistas de BI'
subtitulos_pizza_abi_1 = ['Ferramentas de BI', 'Ferramentas de ETL']
plotar_pizzas(dfs = df_perguntas_abi[:2], titulo_principal=titulo_pizza_abi_1, titulos=subtitulos_pizza_abi_1)

A principal de ferramenta de BI usada por analistas de BI é o Microsoft Power, que supera a quantidade do uso de seus principais concorrentes. O Google Data Studio, Tableau e Pentaho superam separadamente a quantidade do uso de Excel e outras planilhas como ferramentas de BI. Mais da metade do uso de ferramentas de ETL por analistas de BI é feito por SQL e Stored Procedures e Scripts Python.

In [ ]:
# Formatação de valores para o gráfico de pizza
valor_replace_pizza_5 = {'Não utilizo nenhuma das linguagens listadas' : 'Nenhuma'}
df_perguntas_abi[3]['Valor'] = df_perguntas_abi[3]['Valor'].replace(valor_replace_pizza_5)

In [ ]:
# Formatação de valores para o gráfico de pizza
valor_replace_pizza_6 = {'Dados relacionais (estruturados em bancos SQL)' : 'Dados relacionais'}
df_perguntas_abi[2]['Valor'] = df_perguntas_abi[2]['Valor'].replace(valor_replace_pizza_6)

In [ ]:
# Plote de 2 gráficos de pizza para Linguagens e Fontes de Dados usadas por Analistas de BI
titulo_pizza_abi_2 = ' Linguagens e Fontes de Dados usadas por Analistas de BI'
subtitulos_pizza_abi_2 = ['Principal Fonte de Dados', 'Principal Linguagem' ]
plotar_pizzas(dfs = df_perguntas_abi[2:], titulo_principal=titulo_pizza_abi_2, titulos=subtitulos_pizza_abi_2)

Os analistas de BI costumam usar apenas duas fontes de dados, os bancos de dados relacionais e planilhas, e poucas vezes, relativamente falando, usam outra tipo de fonte. A principal linguagem usada é o SQL, em conjunto com os bancos de dados citados anteriormente. A linguagem Python é pouco utilizada se comparada com o SQL, e alguns profissionais não utilizam nenhuma linguagem. 

---
<a id='analista_dados'></a>
### 4.4. Analista de Dados

In [135]:
# Define quais perguntas serão avaliadas para os analistas de dados
perguntas_analise_dados = [
'id',
'Nivel de Ensino',
'Área de Formação',
'Qual sua situação atual de trabalho?',
'Cargo Atual',
'Nivel',
'Faixa salarial',
'Quanto tempo de experiência na área de dados você tem?',
'Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?',
'Quais são os 3 maiores desafios que você tem como gestor no atual momento?',
'Mesmo que esse não seja seu cargo formal, você considera que sua atuação no dia a dia, reflete alguma das opções listadas abaixo?',
'Quais das fontes de dados listadas você já analisou ou processou no trabalho?',
'Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?',
'Quais das linguagens listadas abaixo você utiliza no trabalho?',
'Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?',
'Quais dos bancos de dados/fontes de dados listados abaixo você utiliza no trabalho?',
'Quais das opções de Cloud listadas abaixo você utiliza no trabalho?',
'Quais as Ferramentas de Business Intelligence você utiliza no trabalho?',
'Quais as ferramentas de gestão de Qualidade de dados, Metadados e catálogo de dados você utiliza no trabalho?',
'Em qual das opções abaixo você gasta a maior parte do seu tempo?',
'Quais das opções abaixo fazem parte da sua rotina no trabalho atual com análise de dados?',
'Quais as ferramentas/tecnologias de ETL que você utiliza no trabalho como Data Analyst?',
'Em qual das opções abaixo você gasta a maior parte do seu tempo de trabalho?',
'Quais das opções abaixo fazem parte da sua rotina no trabalho atual com ciência de dados?',
'Quais as técnicas e métodos listados abaixo você costuma utilizar no trabalho?',
'Quais dessas tecnologias fazem parte do seu dia a dia como cientista de dados?'
]

In [137]:
# Filtra os profissionais de Engenharia de dados que trabalham no Brasil
df_analista_dados = df_final.loc[is_analise_dados & is_analista_dados & is_in_Brasil, perguntas_analise_dados]

In [139]:
# Exibe as respostas mais frequentes paras perguntas definidas para os profissionais de Engenharia de dados no Brasil
encontrar_respostas_mais_frequentes(df_analista_dados, valor_desconsiderado='Não Informado')

,Pergunta,Quantidade de Respostas Distintas,Resposta Mais Frequente,Maior Frequência Absoluta,Maior Frequência Relativa (%)
0,Nivel de Ensino,6,Graduação/Bacharelado,105,39.03
1,Área de Formação,9,Computação / Engenharia de Software / Sistemas de Informação/ TI,84,31.58
2,Qual sua situação atual de trabalho?,6,Empregado (CLT),225,83.64
3,Cargo Atual,1,Analista de Dados/Data Analyst,269,100.00
4,Nivel,3,Júnior,102,37.92
5,Faixa salarial,12,(F) R$ (6.001-8.000)/mês,62,23.05
6,Quanto tempo de experiência na área de dados você tem?,7,Menos de 1 ano,67,24.91
7,Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?,7,Não tive experiência na área de TI/Engenharia de Software antes de começar a trabalhar na área de dados,137,50.93
8,"Mesmo que esse não seja seu cargo formal, você considera que sua atuação no dia a dia, reflete alguma das opções listadas abaixo?",1,"*Análise de Dados/BI:* Extrai e cruza dados unindo diferentes fontes da informação; analisa dados visando identificar padrões, gerar insights e levantar perguntas; desenvolve dashboards, relatórios e visualizações de dados em ferramentas de BI.",269,100.00
9,Quais das fontes de dados listadas você já analisou ou processou no trabalho?,75,"Dados relacionais (estruturados em bancos SQL), Planilhas",59,21.93


In [141]:
pergunta_tempo_ad = 'Em qual das opções abaixo você gasta a maior parte do seu tempo de trabalho?'
contar_valores_dummies(df_analista_dados, pergunta_tempo_ad, separador=', ', is_forma_crescente= False, perc_outro=2.0)

,Valor,Quantidade,Porcentagem (%)
4,"Looker, Qlik etc., Realizando construções de dashboards em ferramentas de BI como PowerBI, Tableau",153,33.70
2,Criando consultas através da linguagem SQL para exportar informações e compartilhar com as áreas de negócio.,129,28.41
5,"Processando e analisando dados utilizando linguagens de programação como Python, R etc.",88,19.38
3,Desenvolvendo/cuidando da manutenção de planilhas para atender as áreas de negócio.,37,8.15
0,"Airflow, Dataflow etc., Desenvolvendo/cuidando da manutenção de ETL's utilizando tecnologias como Talend, Pentaho",13,2.86
1,"Atuando na modelagem dos dados, Data Marts etc., com o objetivo de criar conjuntos de dados como Data Warehouses",12,2.64
7,Outros,12,2.64
6,"Realizando experimentos e estudos utilizando metodologias estatísticas como teste de hipótese, modelos de regressão etc.",10,2.20


Os analistas de dados são da mesma área de atuação dos analistas de BI, a área de Análise de Dados/BI, cujos responsabilidades foram citadas anteriormente.

Uma atividade que o analista de dados costuma realizar, mas o analista de BI não, é realizar experimentos e estudos utilizando metodologias estatísticas como teste de hipótese, modelos de regressão etc. As demais atividades dos dois perfis profissionais são muito semelhantes, mas ocorrem em frequências diferentes. O analista de dados tem maior conhecimento em linguagens de programação como o Python e R, e linguagens de consultas SQl do que o analista de BI, concentrando assim, menos tempo na construção de dashboards com ferramentas de BI.


As respostas mais frequentes dos analistas de BI no Brasil em 2021:

- Graduação/Bacharelado em Computação/Engenharia de Software/Sistemas de Informação/TI;
- Não teve experiência na área de TI/Engenharia de Software antes de começar a trabalhar na área de dados;
- Experiência de 2 a 3 anos na área de dados;
- São de nível Júnior;
- Empregados em regime (CLT);
- Salário de (6.001-8.000) reais/mês;
- Dados relacionais (estruturados em bancos SQL) e Planilhas como fontes de dados mais utilizadas.
- Utilizam as linguagens SQL e Python, sendo SQL a mais utilizada;
- Amazon Web Services (AWS) como principal opção de Cloud.

In [143]:
# Plota um mapa de árvore de nível de ensino por nivel de carreira para Analistas de Dados
df_niveis_ad = contar_agrupamento(df_analista_dados, colunas_agrupamento=['Nivel', 'Nivel de Ensino'], coluna_contagem='id')
hierarquia=['Nivel', 'Nivel de Ensino']
df_niveis_ad = df_niveis_ad.sort_values(by=hierarquia, ascending=False)
plotar_treemap_niveis(df_niveis_ad, hierarquia, titulo='Analistas de Dados por Nivel e Nivel de Ensino', raiz='Analista de Dados')

Os três niveis de carreiras de analistas de dados possuem parcelas consideráveis de profissionais com graduação/bacharelado, pós-graduação e mestrado. A maioria dos analistas de dados de nivel júnior e pleno possuem graduação/bacharelado como nivel de ensino, enquanto que a maioria dos de nivel sênior possuem pós-graduação.

In [145]:
# Realiza a contagem de Analistas de Dados por Nivel e Faixa Salarial
df_salario_analista_dados = contar_agrupamento(df_analista_dados, ['Nivel', 'Faixa salarial'], coluna_contagem='id')
df_salario_analista_dados = df_salario_analista_dados.sort_values(by=['Faixa salarial'])

In [147]:
# Removendo o outlier que prejudica a visualização do gráfico
nao_outlier_ad = (df_salario_analista_dados['Nivel'] != 'Junior') & (df_salario_analista_dados['Faixa salarial'] != '(L) R$ (30.001-40.000)/mês')
df_salario_analista_dados = df_salario_analista_dados[nao_outlier_ad]

In [149]:
# Plota áreas das quantidade por faixa salarial por cargo
titulo_area_ad = 'Quantidade de Analistas de Dados por Faixa Salarial e Nivel'
plotar_area(df_salario_analista_dados, eixo_x='Faixa salarial', eixo_y='Quantidade', categorias='Nivel', titulo=titulo_area_ad)

A maioria dos analistas de dados de niveis júnior possuem salário até 6.000 reais por mês, enquanto que os salários dos de nivel pleno variam principalmente entre 3.000 e 12.000 reais por mês, e a maioria dos salários de nivel sênior variam entre 4.000 e 8.000 reais por mês. 

In [151]:
# Define as perguntas cujas respostas são analisadas por gráficos de pizzas
perguntas_pizza_ad = [    
    'Quais as Ferramentas de Business Intelligence você utiliza no trabalho?',
    'Quais as ferramentas/tecnologias de ETL que você utiliza no trabalho como Data Analyst?',
    'Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?',
    'Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?'
]

In [153]:
# Consulta quantidades das respostas para Analistas de Dados
df_perguntas_ad =[]
for p in perguntas_pizza_ad:
    df_perguntas_ad.append(contar_valores_dummies(df_analista_dados, p, separador=', ', is_forma_crescente= False,perc_outro=5.0))

In [155]:
# Formatação de valores para o gráfico de pizza
df_perguntas_ad[0]['Valor'] = df_perguntas_ad[0]['Valor'].replace(valor_replace_pizza_3)

In [157]:
# Formatação de valores para o gráfico de pizza
df_perguntas_ad[1]['Valor'] = df_perguntas_ad[1]['Valor'].replace(valor_replace_pizza_4)

In [159]:
# Plote de 2 gráficos de pizza para ferramentas usadas por Analistas de Dados
titulo_pizza_ad_1 = 'Ferramentas usadas por Analistas de Dados'
subtitulos_pizza_ad_1 = ['Ferramentas de BI', 'Ferramentas de ETL']
plotar_pizzas(dfs = df_perguntas_ad[:2], titulo_principal=titulo_pizza_ad_1, titulos=subtitulos_pizza_ad_1)

As preferências dos analistas de dados para ferramentas de BI e de ETL são muito semelhantes as dos analistas de BI, mudando apenas um pouco os valores das proporções e do uso do Databricks como ferramenta de ETL.

In [161]:
# Formatação de valores para o gráfico de pizza
df_perguntas_ad[2]['Valor'] = df_perguntas_ad[2]['Valor'].replace(valor_replace_pizza_6)

In [163]:
# Formatação de valores para o gráfico de pizza
df_perguntas_ad[3]['Valor'] = df_perguntas_ad[3]['Valor'].replace(valor_replace_pizza_5)
df_perguntas_ad[3]['Valor'] = df_perguntas_ad[3]['Valor'].replace({'Não utilizo nenhuma das linguagens listadas, R' : 'Nenhuma, R'})

In [164]:
# Plote de 2 gráficos de pizza para Linguagens e Fontes de Dados usadas por Analistas de BI
titulo_pizza_ad_2 = ' Linguagens e Fontes de Dados usadas por Analistas de dados'
subtitulos_pizza_ad_2 = ['Principal Fonte de Dados', 'Principal Linguagem' ]
plotar_pizzas(dfs = df_perguntas_ad[2:], titulo_principal=titulo_pizza_ad_2, titulos=subtitulos_pizza_ad_2)

As preferências dos analistas de dados para linguagens e fontes de dados também são muito semelhantes as dos analistas de BI, mudando apenas um pouco os valores das proporções e pelo uso da linguagem R.

<a id='cientista_dados'></a>
### 4.5. Cientista de Dados

In [165]:
# Define as perguntas referentes ao cientista de dados
perguntas_ciencia_dados  = [
'id',
'Nivel de Ensino',
'Área de Formação',
'Qual sua situação atual de trabalho?',
'Cargo Atual',
'Nivel',
'Faixa salarial',
'Quanto tempo de experiência na área de dados você tem?',
'Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?',
'Mesmo que esse não seja seu cargo formal, você considera que sua atuação no dia a dia, reflete alguma das opções listadas abaixo?',
'Quais das fontes de dados listadas você já analisou ou processou no trabalho?',
'Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?',
'Quais das linguagens listadas abaixo você utiliza no trabalho?',
'Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?',
'Quais dos bancos de dados/fontes de dados listados abaixo você utiliza no trabalho?',
'Quais das opções de Cloud listadas abaixo você utiliza no trabalho?',
'Quais as Ferramentas de Business Intelligence você utiliza no trabalho?',
'Quais as ferramentas de gestão de Qualidade de dados, Metadados e catálogo de dados você utiliza no trabalho?',
'Em qual das opções abaixo você gasta a maior parte do seu tempo?',
'Quais as ferramentas/tecnologias de ETL que você utiliza no trabalho como Data Analyst?',
'Em qual das opções abaixo você gasta a maior parte do seu tempo de trabalho?',
'Quais das opções abaixo fazem parte da sua rotina no trabalho atual com ciência de dados?',
'Quais dessas tecnologias fazem parte do seu dia a dia como cientista de dados?',
'Quais as técnicas e métodos listados abaixo você costuma utilizar no trabalho?',
'Em qual das opções abaixo você gasta a maior parte do seu tempo no trabalho?'
]

In [166]:
# Filtra os profissionais de Engenharia de dados que trabalham no Brasil
df_cientista_dados = df_final.loc[is_ciencia_dados & is_cientista_dados & is_in_Brasil, perguntas_ciencia_dados]

In [167]:
# Exibe as respostas mais frequentes paras perguntas definidas para os profissionais de Engenharia de dados no Brasil
encontrar_respostas_mais_frequentes(df_cientista_dados, valor_desconsiderado='Não Informado')

,Pergunta,Quantidade de Respostas Distintas,Resposta Mais Frequente,Maior Frequência Absoluta,Maior Frequência Relativa (%)
0,Nivel de Ensino,6,Graduação/Bacharelado,96,33.57
1,Área de Formação,9,Computação / Engenharia de Software / Sistemas de Informação/ TI,92,32.51
2,Qual sua situação atual de trabalho?,5,Empregado (CLT),240,83.92
3,Cargo Atual,1,Cientista de Dados/Data Scientist,286,100.00
4,Nivel,3,Pleno,127,44.41
5,Faixa salarial,13,(G) R$ (8.001-12.000)/mês,71,24.83
6,Quanto tempo de experiência na área de dados você tem?,6,de 2 a 3 anos,89,31.12
7,Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?,7,Não tive experiência na área de TI/Engenharia de Software antes de começar a trabalhar na área de dados,122,42.66
8,"Mesmo que esse não seja seu cargo formal, você considera que sua atuação no dia a dia, reflete alguma das opções listadas abaixo?",1,*Ciência de Dados/Machine Learning: *Desenha e executa experimentos com o objetivo de responder perguntas do negócio; desenvolve modelos preditivos e algoritmos de Machine Learning com o objetivo de otimizar e automatizar a tomada de decisão.,286,100.00
9,Quais das fontes de dados listadas você já analisou ou processou no trabalho?,110,"Dados relacionais (estruturados em bancos SQL), Planilhas",36,12.63


In [168]:
pergunta_cd_1 = 'Quais das opções abaixo fazem parte da sua rotina no trabalho atual com ciência de dados?'
contar_valores_dummies(df_cientista_dados, pergunta_cd_1, separador=', ', is_forma_crescente= False,perc_outro=5.0)

,Valor,Quantidade,Porcentagem (%)
2,Desenvolvo modelos de Machine Learning com o objetivo de colocar em produção em sistemas (produtos de dados).,223,18.72
3,"Estudos Ad-hoc com o objetivo de confirmar hipóteses, análise de cluster para resolver problemas pontuais e responder perguntas das áreas de negócio., forecasts, realizar modelos preditivos",217,18.22
5,Sou responsável pela coleta e limpeza dos dados que uso para análise e modelagem.,192,16.12
6,"Sou responsável por entrar em contato com os times de negócio para definição do problema, identificar a solução e apresentação de resultados.",183,15.37
1,"Cuido da manutenção de modelos de Machine Learning já em produção, ajustes e refatoração quando necessário., atuando no monitoramento",105,8.82
7,Outros,101,8.48
0,"APIs de consumo e monitoramento., Sou responsável por colocar modelos em produção, criar os pipelines de dados",100,8.40
4,"Looker, Qlik, Realizo construções de dashboards em ferramentas de BI como PowerBI, Tableau, etc.",70,5.88


In [169]:
pergunta_cd_2 = 'Quais as técnicas e métodos listados abaixo você costuma utilizar no trabalho?'
contar_valores_dummies(df_cientista_dados, pergunta_cd_2, separador=', ', is_forma_crescente= False,perc_outro=0.0)

,Valor,Quantidade,Porcentagem (%)
2,"GLM)., Utilizo modelos de regressão (linear, logística",226,16.28
11,Utilizo redes neurais ou modelos baseados em árvore para criar modelos de classificação.,204,14.70
10,"Utilizo métodos estatísticos clássicos (Testes de hipótese, análise multivariada, dados longitudinais, inferência estatística) para analisar dados., sobrevivência",175,12.61
0,"DBScan etc)., Spectral, Utilizo técnicas de Clusterização (K-means",174,12.54
3,Realizo previsões através de modelos de Séries Temporais (Time Series).,135,9.73
12,Utilizo técnicas de NLP (Natural Language Processing) para análisar dados não-estruturados.,108,7.78
5,Utilizo modelos de Detecção de Churn.,83,5.98
9,Utilizo métodos estatísticos Bayesianos para analisar dados.,76,5.48
6,Utilizo modelos de Machine Learning para detecção de fraude.,75,5.40
1,Desenvolvo sistemas de recomendação (RecSys).,52,3.75


In [170]:
pergunta_cd_3 = 'Quais dessas tecnologias fazem parte do seu dia a dia como cientista de dados?'
contar_valores_dummies(df_cientista_dados, pergunta_cd_3, separador=', ', is_forma_crescente= False,perc_outro=0.0)

,Valor,Quantidade,Porcentagem (%)
2,"Ambientes de desenvolvimento local (R-studio, Anaconda)., JupyterLab",250,20.92
8,"Google Sheets etc)., Planilhas (Excel",191,15.98
4,"DVC, Gitlab etc)., Neptune, Sistemas de controle de versão (Github",186,15.56
1,"AWS Sagemaker, Ambientes de desenvolvimento na nuvem (Google Colab, Kaggle Notebooks etc).",161,13.47
5,"Ferramentas de BI (PowerBI, Looker, Qlik etc)., Tableau",114,9.54
3,"Azure Machine Learning, Kubeflow etc)., Plataformas de Machine Learning (TensorFlow",113,9.46
9,"Plataformas de Data Apps (Streamlit, Plotly Dash etc)., Shiny",65,5.44
6,"Ferramentas de ETL (Apache Airflow, Fivetran, NiFi, Pentaho etc)., Stitch",59,4.94
0,"AWS Feature Store, Auto-Keras etc)., Databricks Feature Store etc)., Feature Store (Feast, Ferramentas de AutoML (Datarobot, H2O, Hopsworks",29,2.43
7,"Ferramentas de estatística avançada como SPSS, SAS etc.",27,2.26


A área de Ciência de Dados/Machine Learning é, de forma geral, caracterizada pelas responsabilidades de: 
- Desenhar e executa experimentos com o objetivo de responder perguntas do negócio. 
- Desenvolver modelos preditivos e algoritmos de Machine Learning com o objetivo de otimizar e automatizar a tomada de decisão.


As tarefas mais comuns da rotina de um cientista de dados, daquelas com maiores frequências para a de menores frequências, são:
- Desenvolver modelos de Machine Learning com o objetivo de colocar em produção em sistemas (produtos de dados).
- Realizar estudos Ad-hoc com o objetivo de confirmar hipóteses, criar modelos preditivos e forecasts, analisar cluster para resolver problemas pontuais e responder perguntas das áreas de negócio. 
- Coletar e limpar os dados que uso para análise e modelagem.
- Entrar em contato com os times de negócio para definição do problema, identificar a solução e apresentação de resultados.
- Cuidar da manutenção de modelos de Machine Learning já em produção, ajustes e refatoração quando necessário., atuando no monitoramento.
- Colocar modelos em produção, criar os pipelines de dados, APIs de consumo e monitoramento.
- Realizar construções de dashboards em ferramentas de BI como PowerBI, Tableau, Looker, Qlik, etc.


As principais tecnologias usadas no trabalho dos cientistas de dados são:
- JupyterLab e Ambientes de desenvolvimento local (R-studio, Anaconda). 	
- Planilhas (ExcelGoogle Sheets etc).
- Sistemas de controle de versão (Github, Neptune, DVC, Gitlab etc).
- Ambientes de desenvolvimento na nuvem (Google Colab, AWS Sagemaker, Kaggle Notebooks etc).	
- Ferramentas de BI (PowerBI, Tableau, Looker, Qlik etc)., 	
- Plataformas de Machine Learning (TensorFlow, Azure Machine Learning, Kubeflow etc).


As técnicas e métodos mais comuns usadas por cientistas de dados são:
- Utilizar modelos de regressão (linear, logística, GLM).
- Utilizar redes neurais ou modelos baseados em árvore para criar modelos de classificação.	
- Utilizar métodos estatísticos clássicos (Testes de hipótese, análise multivariada, dados longitudinais, inferência estatística) para analisar dados.
- Utilizar técnicas de Clusterização (K-means, Spectral, DBScan etc).
- Realizar previsões através de modelos de Séries Temporais (Time Series).
- Utilizar técnicas de NLP (Natural Language Processing) para análisar dados não-estruturados.
- Utilizar modelos de Detecção de Churn.
- Utilizar métodos estatísticos Bayesianos para analisar dados.
- Utilizar modelos de Machine Learning para detecção de fraude.
- Desenvolver sistemas de recomendação (RecSys).
- Utilizar métodos de Visão Computacional.
- Utilizar modelos de Reinforcement Learning (aprendizado por reforço).	
- Utilizar cadeias de Markov ou HMM's para realizar análises de dados.	


As respostas mais frequentes dos profissionais de Ciência de Dados/Machine Learning no Brasil em 2021:




In [171]:
# Plota um mapa de árvore de nível de ensino por nivel de carreira para Cientistas de Dados
df_niveis_cd = contar_agrupamento(df_cientista_dados, colunas_agrupamento=['Nivel', 'Nivel de Ensino'], coluna_contagem='id')
hierarquia=['Nivel', 'Nivel de Ensino']
df_niveis_cd = df_niveis_cd.sort_values(by=hierarquia, ascending=False)
plotar_treemap_niveis(df_niveis_cd, hierarquia, titulo='Cientistas de Dados por Nivel e Nivel de Ensino', raiz='Cientista de dados')

Todos os niveis de cientista de dados possuem significativas quantidades de profissionais com mestrado ou doutorado ou phd. Mestrado é o nivel de ensino mais frequente para profissionais sêniores. A graduação/bacharelado é o nivel de ensino mais frequente para cientistas de dados de niveis júnior e pleno.

In [172]:
# Realiza a contagem de Cientista de Dados por Nivel e Faixa Salarial
df_salario_cientista_dados = contar_agrupamento(df_cientista_dados, ['Nivel', 'Faixa salarial'], coluna_contagem='id')
df_salario_cientista_dados = df_salario_cientista_dados.sort_values(by=['Faixa salarial'])

In [173]:
# Plota área da quantidade por faixa salarial por cargo
titulo_area_cd = 'Quantidade de Cientistas de Dados por Faixa Salarial e Nivel'
plotar_area(df_salario_cientista_dados, eixo_x='Faixa salarial', eixo_y='Quantidade', categorias='Nivel', titulo=titulo_area_cd)

A maioria dos cientistas de dados de nivel júnior ganham até 8.000 reais por mês, enquanto o de nivel pleno tendem a ganhar entre 4.000 e 16.000 reais por mês, e os de nivel sênior 6.000 e 20.000 reais por mês.

In [174]:
# Define as perguntas cujas respostas são analisadas por gráficos de pizzas
perguntas_pizza_cd = [    
    'Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?',
    'Quais as técnicas e métodos listados abaixo você costuma utilizar no trabalho?'
]

In [175]:
# Consulta quantidades das respostas 
df_perguntas_cd =[]
for p in perguntas_pizza_cd:
    df_perguntas_cd.append(contar_valores_dummies(df_cientista_dados, p, separador=', ', is_forma_crescente= False,perc_outro=5.0))

In [176]:
# Formatação de valores para o gráfico de pizza
valor_replace_pizza_100 = {'GLM)., Utilizo modelos de regressão (linear, logística': 'Regressão',
       'Utilizo redes neurais ou modelos baseados em árvore para criar modelos de classificação.' : 'Classificação',
       'Utilizo métodos estatísticos clássicos (Testes de hipótese, análise multivariada, dados longitudinais, inferência estatística) para analisar dados., sobrevivência' : 'Estatística',
       'DBScan etc)., Spectral, Utilizo técnicas de Clusterização (K-means' : 'Clusterização',
       'Realizo previsões através de modelos de Séries Temporais (Time Series).' : 'Séries Temporais',
       'Outros' : 'Outros',
       'Utilizo técnicas de NLP (Natural Language Processing) para análisar dados não-estruturados.' : 'NLP',
       'Utilizo modelos de Detecção de Churn.' : 'Detecção de Churn',
       'Utilizo métodos estatísticos Bayesianos para analisar dados.' : 'Estatística Bayesiana',
       'Utilizo modelos de Machine Learning para detecção de fraude.' : 'Detecção de fraude'}

df_perguntas_cd[1]['Valor'] = df_perguntas_cd[1]['Valor'].replace(valor_replace_pizza_100)

In [177]:
titulo_pizza_cd_1 = ' Linguagens, Técnicas e Métodos usados por Cientistas de Dados'
subtitulos_pizza_cd_1 = ['Principal Linguagem', 'Principais técnicas e métodos']
plotar_pizzas(dfs = df_perguntas_cd[:2], titulo_principal=titulo_pizza_cd_1, titulos=subtitulos_pizza_cd_1)

Python é a linguagem de programação mais usada por cientistas de dados, representando 81,4% das respostas, contra apenas 7,72% da sua principal concorrente, a linguagem R. A linguagem de consulta SQL se mostra muito importante, sendo escolhida por 9,82% dos profissionais de um cargo que tende a fazer menos consultas do que os analisados anteriormente.

A distribuição das respostas das técnicas e métodos usadas pelos cientistas de dados não mostra um consenso, mas sim uma ampla gama de conhecimentos em machine learning que o profissional teria que dominar para poder trabalhar em qualquer vaga do cargo. Algumas vezes o profissional atua em uma área bem específica e terá que dominar apenas algumas dessas técnicas de forma plena. As respostas das técnicas e métodos usadas foram combinações dos valores mostrados.

---
<a id='machine_learning'></a>
## 5. Aprendizado de máquina

---
<a id='pre_processing'></a>
### 5.1. Pré-Processamento

Na etapa de Pré-Processamento, apenas algumas variáveis serão selecionadas, os dados serão filtrados segundos alguns critétrios e codificados para valores numéricos

In [178]:
# Cria filtro para a coluna 'Quanto tempo de experiência na área de dados você tem?'
possui_experiencia = df_final['Quanto tempo de experiência na área de dados você tem?'] != 'Não tenho experiência na área de dados'

In [179]:
# Cria filtro para a coluna 'Nivel de Ensino'
possui_nivel_ensino = (df_final['Nivel de Ensino'] !='Prefiro não informar') & (df_final['Nivel de Ensino'] !='Não tenho graduação formal')

In [180]:
# Cria filtro para a coluna 'Faixa salarial'
possui_salario_adequado = (df_final['Faixa salarial'] != '(A) Menos de R$ 1.000/mês')

In [181]:
# Cria o dataframe df_ml a partir de df_final para o aprendizado de máquina
linhas_ml = (is_cargo_analisado & possui_nivel_ensino & possui_experiencia & is_in_Brasil & possui_salario_adequado)
colunas_ml = ['Faixa salarial', 'Cargo Atual', 'Nivel', 'Quanto tempo de experiência na área de dados você tem?']

df_ml = df_final.loc[linhas_ml, colunas_ml].copy()

In [182]:
# Exibe as dimensões df_ml
print_dimensoes_dataframe(df_ml, nome='DataFrame de Machine Learning')

As dimensões do dataframe DataFrame de Machine Learning são 1239 linhas e 4 colunas.


In [183]:
# Renomeia a coluna 'Quanto tempo de experiência na área de dados você tem?' para 'Experiência'
df_ml.rename(columns={'Quanto tempo de experiência na área de dados você tem?': 'Experiência'}, inplace=True)

In [184]:
# Exibe as 5 primeiras linhas de df_ml
df_ml.head()

,Faixa salarial,Cargo Atual,Nivel,Experiência
482,(F) R$ (6.001-8.000)/mês,Engenheiro de Dados/Data Engineer,Sênior,Menos de 1 ano
483,(F) R$ (6.001-8.000)/mês,Cientista de Dados/Data Scientist,Pleno,de 6 a 10 anos
485,(E) R$ (4.001-6.000)/mês,Analista de BI/BI Analyst/Analytics Engineer,Pleno,de 1 a 2 anos
486,(F) R$ (6.001-8.000)/mês,Engenheiro de Dados/Data Engineer,Pleno,de 4 a 5 anos
487,(D) R$ (3.001-4.000)/mês,Analista de BI/BI Analyst/Analytics Engineer,Júnior,de 1 a 2 anos


In [185]:
# Conta os valores de 'Faixa salarial'
contar_valores(df_ml['Faixa salarial'])

,Quantidade,Porcentagem (%)
(G) R$ (8.001-12.000)/mês,262,21.15
(E) R$ (4.001-6.000)/mês,258,20.82
(F) R$ (6.001-8.000)/mês,253,20.42
(H) R$ (12.001-16.000)/mês,112,9.04
(D) R$ (3.001-4.000)/mês,111,8.96
(C) R$ (2.001-3.000)/mês,101,8.15
(B) R$ (1.001-2.000)/mês,70,5.65
(I) R$ (16.001-20.000)/mês,35,2.82
(J) R$ (20.001-25.000)/mês,13,1.05
(L) R$ (30.001-40.000)/mês,12,0.97


In [186]:
# Cria um dicionário para os valores de 'Faixa Salarial' para dígitos
codificacao_faixa_salarial = {
    #'(A) Menos de R$ 1.000/mês' : '0',
    '(B) R$ (1.001-2.000)/mês' : '0',
    '(C) R$ (2.001-3.000)/mês' : '0',
    '(D) R$ (3.001-4.000)/mês' : '0',
    '(E) R$ (4.001-6.000)/mês' : '1', 
    '(F) R$ (6.001-8.000)/mês' : '2',
    '(G) R$ (8.001-12.000)/mês' : '3',
    '(H) R$ (12.001-16.000)/mês' : '3',
    '(I) R$ (16.001-20.000)/mês' : '3',
    '(J) R$ (20.001-25.000)/mês' : '3',
    '(K) R$ (25.001-30.000)/mês' : '3',
    '(L) R$ (30.001-40.000)/mês' : '3',
    '(M) Acima de R$ 40.001/mês' : '3'
}

In [187]:
# Cria um dicionário para os valores de 'Cargo Atual' para dígitos
codificacao_cargo = {
    'Analista de BI/BI Analyst/Analytics Engineer' : '0',
    'Analista de Dados/Data Analyst' : '1',
    'Cientista de Dados/Data Scientist' : '2',
    'Engenheiro de Dados/Data Engineer' : '3'
}

In [188]:
# Cria um dicionário para os valores de 'Nivel' para dígitos
codificacao_nivel = {
    'Júnior' : '0',
    'Pleno' : '1',
    'Sênior' : '2'
}

In [189]:
# Cria um dicionário para os valores de 'Experiência' para dígitos
codificacao_experiencia = {
    'Menos de 1 ano' : '0',
    'de 1 a 2 anos' : '1',
    'de 2 a 3 anos' : '2',
    'de 4 a 5 anos' : '3',
    'de 6 a 10 anos' : '4',
    'Mais de 10 anos' : '4'
}

In [190]:
# Definição da função codificar_serie()
def codificar_serie(serie, codificacao):
    """
    Retorna uma série de valores inteiros codificada em dígitos de acordo com codificacao
    
    Argumentos:
    serie -- Serie
    codificacao -- dict
    """
    serie = serie.replace(codificacao)
    return serie.astype('int64')

In [191]:
# Converte a coluna 'Faixa salarial' para o tipo int64 (inteiro)
df_ml['Faixa salarial'] = codificar_serie(df_ml['Faixa salarial'], codificacao_faixa_salarial)

In [192]:
# Verifica se a coluna 'Faixa salarial' é do tipo int64 (inteiro)
assert np.int64 == df_ml['Faixa salarial'].dtype

In [193]:
# Converte a coluna 'Cargo Atual' para o tipo int64 (inteiro)
df_ml['Cargo Atual'] = codificar_serie(df_ml['Cargo Atual'], codificacao_cargo)

In [194]:
# Verifica se a coluna 'Cargo Atual' é do tipo int64 (inteiro)
assert np.int64 == df_ml['Cargo Atual'].dtype

In [195]:
# Converte a coluna 'Nivel' para o tipo int64 (inteiro)
df_ml['Nivel'] = codificar_serie(df_ml['Nivel'], codificacao_nivel)

In [196]:
# Verifica se a coluna 'Nivel' é do tipo int64 (inteiro)
assert np.int64 == df_ml['Nivel'].dtype

In [197]:
# Converte a coluna'Experiência' para o tipo int64 (inteiro)
df_ml['Experiência'] = codificar_serie(df_ml['Experiência'], codificacao_experiencia)

In [198]:
# Verifica se a coluna 'Experiência' é do tipo int64 (inteiro)
assert np.int64 ==df_ml['Experiência'].dtype

---
<a id='model_train'></a>
### 5.2. Treinamento do modelo

O modelo de aprendizado de máquina será de classificação, onde cada classe salarial será uma classe, e serão considerados os valores do cargo, nivel de carreira e tempo de experiência na área de dados como recursos (features). Algumas das faixas salariais são diferentes das analisadas anteriormente, já que é necessário manter uma certa proximidade entre a quantidade para cada classe.

O algoritmo escolhido é um baseado em estruturas de árvores e teve sua escolha de hiperparâmetros otimizado por uma pesquisa em grade (Grid Search). 

In [199]:
# Divisão entre variáveis preditoras (features) e predita (target)
X, y = df_ml.drop(columns=['Faixa salarial']), df_ml['Faixa salarial']

In [200]:
# Conta os valores codificados de 'Cargo Atual'
contar_valores(df_ml['Cargo Atual'])

,Quantidade,Porcentagem (%)
2,341,27.52
0,320,25.83
1,299,24.13
3,279,22.52


In [201]:
# Conta os valores codificados de 'Faixa salarial'
contar_valores(df_ml['Faixa salarial'])

,Quantidade,Porcentagem (%)
3,446,36.00
0,282,22.76
1,258,20.82
2,253,20.42


In [202]:
# Conta os valores codificados de 'Nivel'
contar_valores(df_ml['Nivel'])

,Quantidade,Porcentagem (%)
1,470,37.93
0,411,33.17
2,358,28.89


In [203]:
# Conta os valores codificados de'Experiência'
contar_valores(df_ml['Experiência'])

,Quantidade,Porcentagem (%)
2,336,27.12
1,264,21.31
0,231,18.64
3,214,17.27
4,194,15.66


In [204]:
# Cria um classificador baseado em Árvore de Decisão
classificador_base = DecisionTreeClassifier(random_state=RANDOM_STATE)

In [205]:
# Define o espaço de busca por hiperparâmetros
possiveis_hiperparametros = {
    'max_depth' : [2, 3, 4, 5, 6, 7, 8],
    'min_samples_leaf' : [0.03, 0.05, 0.07, 0.1, 0.13, 0.15]
}

In [206]:
# Cria o otimizador de hiperparâmetros
otimizador_hiperparametros = GridSearchCV(estimator=classificador_base,
                       param_grid=possiveis_hiperparametros,
                       scoring='accuracy',
                       cv=5,
                       n_jobs=-1)

In [207]:
# Treina o otimizador em busca dos melhores hiperparâmetros
otimizador_hiperparametros.fit(X, y);

In [208]:
# Exibe os melhores hiperparâmetros encontrados pelo otimizador
print('Os melhores hiperparâmetros para o modelo são:\n', otimizador_hiperparametros.best_params_)

Os melhores hiperparâmetros para o modelo são:
 {'max_depth': 5, 'min_samples_leaf': 0.05}


In [209]:
# Encontra e exibe a melhor acurácia encontrada nas validações cruzadas
melhor_acuracia = otimizador_hiperparametros.best_score_
melhor_acuracia = round(melhor_acuracia * 100, 2)
print('A melhor acurácia no treinamento (validação cruzada) foi {}%.'.format(melhor_acuracia))

A melhor acurácia no treinamento (validação cruzada) foi 57.95%.


In [210]:
# Cria o modelo com melhores hiperparâmetros encontrados pelo otimizador
melhor_classificador = otimizador_hiperparametros.best_estimator_

In [211]:
# Definição da função plotar_arvore_decisao()
def plotar_arvore_decisao(classificador, nomes_features, nomes_classes):
    """
    Plota uma árvore de decisão.
    
    Argumentos:
    classificador -- DecisionTreeClassifier
    nomes_features -- list
    nomes_classes -- list
    """
    plt.figure(figsize = (20,7), dpi=900)
    tree.plot_tree(classificador,
                   feature_names = nomes_features, 
                   class_names=nomes_classes,
                   filled = True
                   );
    #fig.savefig('arvore.png')
    plt.show()

In [212]:
# Atribui os nomes das features do modelo
nomes_features = X.columns.to_list()
nomes_features

['Cargo Atual', 'Nivel', 'Experiência']

In [213]:
# Atribui os nomes das classes do modelo
nomes_classes = [
    'F1 - Até R$ 4.000/mês',
    'F2 - R$ 4.001-6.000 / mês',
    'F3 - R$ 6.001-8.000 / mês',
    'F4 - R$ 8.001-12.000 / mês',
    'F5 - Mais de R$ 12.001/mês'    
]

In [214]:
# Plota a árvore de decisão do melhor modelo de classificação
#plotar_arvore_decisao(melhor_classificador, nomes_features, nomes_classes)

In [215]:
# Definição da função salvar_modelo_ml()
def salvar_modelo_ml(modelo, arquivo='Modelo-ML'):
    """
    Salva um modelo de machine learning em um arquivo.
    
    Argumentos:
    modelo -- string
    arquivo -- string (Opcional - 'Modelo-ML')
    """
    arquivo = arquivo + '.pkl'
    joblib.dump(modelo, arquivo)

In [216]:
# Salva o melhor modelo em um arquivo
salvar_modelo_ml(melhor_classificador, arquivo='Classificador-FaixaSalarial-2021')

---
<a id='predictions'></a>
### 5.3. Predição

A principal contribuição do modelo treinado anteriormente para o projeto é prever a faixa salarial mais provável e os valores de probabilidades para cada faixa salarial dado os perfis mais prováveis dos profissionais considerados. Com isso, é possível auxiliares candidatos e empregadores a avaliarem as propostas e pretensões salariais dos profissionais de dados no Brasil, ainda que a pesquisa tenha sido feita em 2021.

In [217]:
# Define o nome dos perfis das classes das previsões
perfis_index = [
                # Perfis de Analistas de BI            
                'Analista de BI - Júnior - Experiência de menos de 1 ano',
                'Analista de BI - Júnior - Experiênciade 1 a 2 anos',
                'Analista de BI - Pleno - Experiência de 1 a 2 anos',
                'Analista de BI - Pleno - Experiência de 2 a 3 anos',
                'Analista de BI - Sênior - Experiência de 2 a 3 anos',
                'Analista de BI - Sênior - Experiência de 4 a 5 anos',
                'Analista de BI - Sênior - Experiência de mais de 6 anos',
    
                # Perfis de Analistas de Dados
                'Analista de Dados - Júnior - Experiência de menos de 1 ano',
                'Analista de Dados - Júnior - Experiência de 1 a 2 anos',
                'Analista de Dados - Pleno - Experiência de 1 a 2 anos',
                'Analista de Dados - Pleno - Experiência de 2 a 3 anos',
                'Analista de Dados - Sênior - Experiência de 2 a 3 anos',
                'Analista de Dados - Sênior - Experiência de 4 a 5 anos',
                'Analista de Dados - Sênior - Experiência de mais de 6 anos',
    
                # Perfis de Cientistas de Dados
                'Cientista de Dados - Júnior - Experiência de menos de 1 ano',
                'Cientista de Dados - Júnior - Experiência de 1 a 2 anos',
                'Cientista de Dados - Pleno - Experiência de 1 a 2 anos',
                'Cientista de Dados - Pleno - Experiência de 2 a 3 anos',
                'Cientista de Dados - Sênior - Experiência de 2 a 3 anos',
                'Cientista de Dados - Sênior - Experiência de 4 a 5 anos',
                'Cientista de Dados - Sênior - Experiência de mais de 6 anos',
    
                # Perfis de Engenheiros de Dados
                'Engenheiro de Dados - Júnior - Experiência de menos de 1 ano',
                'Engenheiro de Dados - Júnior - Experiência de 1 a 2 anos',
                'Engenheiro de Dados - Pleno - Experiência de 1 a 2 anos',
                'Engenheiro de Dados - Pleno - Experiência de 2 a 3 anos',
                'Engenheiro de Dados - Sênior - Experiência de 2 a 3 anos',
                'Engenheiro de Dados - Sênior - Experiência de 4 a 5 anos',
                'Engenheiro de Dados - Sênior - Experiência de mais de 6 anos'
               ]

In [218]:
# Cria perfis com 'Cargo Atual', 'Nivel', 'Experiência'
perfis = [
            # Perfis de Analistas de BI 
            [0,0,0],
            [0,0,1],
            [0,1,1],
            [0,1,2],
            [0,2,2],
            [0,2,3],
            [0,2,4],
    
            # Perfis de Analistas de Dados
            [1,0,0],
            [1,0,1],
            [1,1,1],
            [1,1,2],
            [1,2,2],
            [1,2,3],
            [1,2,4],
    
            # Perfis de Cientistas de Dados
            [2,0,0],
            [2,0,1],
            [2,1,1],
            [2,1,2],
            [2,2,2],
            [2,2,3],
            [2,2,4],

            # Perfis de Engenheiros de Dados
            [3,0,0],
            [3,0,1],
            [3,1,1],
            [3,1,2],
            [3,2,2],
            [3,2,3],
            [3,2,4]
        ]

In [219]:
# Realiza predição de 'Faixa Salarial' por perfil
X_pred = pd.DataFrame(perfis, columns = nomes_features , dtype = int)
df_pred = pd.DataFrame(melhor_classificador.predict(X_pred), columns=['Faixa Salarial Prevista'])

In [220]:
# Define a decodificação de valores de 'Faixa Salarial'
faixa_salarial_decodificacao = {
    '0' : 'Faixa 1 - Até R$ 4.000/mês',
    '1' : 'Faixa 2 - R$ 4.001-6.000/mês',
    '2' : 'Faixa 3 - R$ 6.001-8.000/mês',
    '3' : 'Faixa 4 - R$ 8.001-12.000/mês' 
}

In [221]:
# Converte e decodifica os valores de 'Faixa Salarial'
df_pred['Faixa Salarial Prevista'] = df_pred['Faixa Salarial Prevista'].astype(str)
df_pred['Faixa Salarial Prevista'] = df_pred['Faixa Salarial Prevista'].replace(faixa_salarial_decodificacao)

In [222]:
# Define os nomes das colunas de probabilidade de previsão por classe
colunas_prob = ['Probabilidade (%) de salário até R$ 4.000/mês', 
                'Probabilidade (%) de salário de R$ 4.001-6.000/mês',
                'Probabilidade (%) de salário de R$ 6.001-8.000/mês', 
                'Probabilidade (%) de salário maior que R$ 8.000/mês']

In [223]:
# Cria dataframe com as probabilidades de previsões para cada classe
df_probs = pd.DataFrame(melhor_classificador.predict_proba(X_pred), columns=colunas_prob)
converte_probabilidade = lambda x : (x * 100).round(2)
df_probs = df_probs.apply(converte_probabilidade)

In [224]:
# Une os dataframes de previsões e de probabilidades de previsões para cada classe
df_pred_probs = pd.concat([df_pred, df_probs], axis=1)
#df_pred_probs.index = ['Perfil ' + str(i+1) for i in range(df_pred_probs.shape[0])]
df_pred_probs.index = perfis_index
df_pred_probs = df_pred_probs.reset_index()
df_pred_probs = df_pred_probs.rename(columns={'index': 'Perfil'})
df_pred_probs

,Perfil,Faixa Salarial Prevista,Probabilidade (%) de salário até R$ 4.000/mês,Probabilidade (%) de salário de R$ 4.001-6.000/mês,Probabilidade (%) de salário de R$ 6.001-8.000/mês,Probabilidade (%) de salário maior que R$ 8.000/mês
0,Analista de BI - Júnior - Experiência de menos de 1 ano,Faixa 1 - Até R$ 4.000/mês,76.53,16.33,4.08,3.06
1,Analista de BI - Júnior - Experiênciade 1 a 2 anos,Faixa 1 - Até R$ 4.000/mês,76.53,16.33,4.08,3.06
2,Analista de BI - Pleno - Experiência de 1 a 2 anos,Faixa 2 - R$ 4.001-6.000/mês,21.88,45.31,25.00,7.81
3,Analista de BI - Pleno - Experiência de 2 a 3 anos,Faixa 2 - R$ 4.001-6.000/mês,11.63,46.51,25.58,16.28
4,Analista de BI - Sênior - Experiência de 2 a 3 anos,Faixa 4 - R$ 8.001-12.000/mês,1.16,11.63,19.77,67.44
5,Analista de BI - Sênior - Experiência de 4 a 5 anos,Faixa 4 - R$ 8.001-12.000/mês,1.16,11.63,19.77,67.44
6,Analista de BI - Sênior - Experiência de mais de 6 anos,Faixa 4 - R$ 8.001-12.000/mês,1.16,11.63,19.77,67.44
7,Analista de Dados - Júnior - Experiência de menos de 1 ano,Faixa 1 - Até R$ 4.000/mês,65.59,25.81,6.45,2.15
8,Analista de Dados - Júnior - Experiência de 1 a 2 anos,Faixa 1 - Até R$ 4.000/mês,65.59,25.81,6.45,2.15
9,Analista de Dados - Pleno - Experiência de 1 a 2 anos,Faixa 2 - R$ 4.001-6.000/mês,21.88,45.31,25.00,7.81


In [225]:
# Salva os dados de predições em um arquivo
salvar_df_csv(df_pred_probs, arquivo='Previsoes_salarios_perfis_2021')

---
<a id='references'></a>
### Referências

UDACITY - Data Analyst Nanodegree Program: https://www.udacity.com/course/data-analyst-nanodegree--nd002

KAGGLE - Intermediate Machine Learning:  https://www.kaggle.com/learn/intermediate-machine-learning

---

<div style="text-align: center"> Copyright &copy; 2022 | Leonardo Simões | All Rights Reserved </div>

---